In [ ]:
#Run every cell, in order, until instructed otherwise

In [1]:
import pandas as pd
import numpy as npy
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import random
import sklearn
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
model=LogisticRegression()
import itertools

In [2]:
#this cell contains all functions defined for building columns in ufcfightscrap
#converts from '%B %d, %Y' (i.e. August 22, 2020) to date (i.e. 2020-08-22)
def convert_to_datetime(day1):
    return datetime.strptime(day1, '%B %d, %Y').date()
convert_to_datetime_vect= npy.vectorize(convert_to_datetime)

#this age function is written in such a stupid way
def age(birthDate,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    if birthDate=='--':
        aa='unknown'
    elif type(birthDate)==str and not type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif type(birthDate)==str and type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif not type(birthDate)==str and type(day)==str:
        bd=birthDate
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    else:
        bd=birthDate
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    return aa

age_vect= npy.vectorize(age)

def fighter_age(fighter,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            dob=datetime.strptime(ufcfighterscrap['dob'][i], '%b %d, %Y').strftime('%B %d, %Y')
            a=age(dob,day,form1,form2)
            break
    return a

def wins_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
    return summ

def losses_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
    return summ

#functions for height and reach
def fighter_height(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['height'][i]
            break
    if a=='--':
        b='unknown'
    elif a[4]=='"':
        b=int(a[0])*30.48+int(a[3])*2.54
    else:
        b=int(a[0])*30.48+int(a[3]+a[4])*2.54
    return b

def fighter_reach(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['reach'][i]
            break
    if a=='--':
        b='unknown'
    else:
        b=int(a[0]+a[1])*2.54
    return b

wins_before_vect= npy.vectorize(wins_before)
losses_before_vect= npy.vectorize(losses_before)
fighter_height_vect= npy.vectorize(fighter_height)
fighter_reach_vect= npy.vectorize(fighter_reach)

#day1 should be input in the form '%B %d, %Y' i.e. 'August 20, 1962'
#conversions can be made via day=datetime.strptime(ufcfightsML_known_df['date'][i], '%B %d, %Y').strftime('%b %d, %Y')
def time_diff(day1,day2=date.today()):
    if day2==date.today():
        answer=(day2-datetime.strptime(day, '%B %d, %Y')).days
    else:
        answer=(datetime.strptime(day2, '%B %d, %Y')-datetime.strptime(day1, '%B %d, %Y')).days
    return answer

#we now vectorize this to use in pandas/numpy
time_diff_vect= npy.vectorize(time_diff)

#can make a single function to do all of these... actually maybe the count function would even work as is

def L5Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L5Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L2Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def L2Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return summ

def ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return summ

def L5Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L5Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L2Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def L2Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return summ

def sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return summ

def L5Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L5Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L2Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def L2Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

#vectorize all of these functions
L5Y_wins_vect= npy.vectorize(L5Y_wins)
L5Y_losses_vect= npy.vectorize(L5Y_losses)
L2Y_wins_vect= npy.vectorize(L2Y_wins)
L2Y_losses_vect= npy.vectorize(L2Y_losses)
ko_wins_vect= npy.vectorize(ko_wins)
ko_losses_vect= npy.vectorize(ko_losses)
L5Y_ko_wins_vect= npy.vectorize(L5Y_ko_wins)
L5Y_ko_losses_vect= npy.vectorize(L5Y_ko_losses)
L2Y_ko_wins_vect= npy.vectorize(L2Y_ko_wins)
L2Y_ko_losses_vect= npy.vectorize(L2Y_ko_losses)
sub_wins_vect= npy.vectorize(sub_wins)
sub_losses_vect= npy.vectorize(sub_losses)
L5Y_sub_wins_vect= npy.vectorize(L5Y_sub_wins)
L5Y_sub_losses_vect= npy.vectorize(L5Y_sub_losses)
L2Y_sub_wins_vect= npy.vectorize(L2Y_sub_wins)
L2Y_sub_losses_vect= npy.vectorize(L2Y_sub_losses)

#for columns like fighter_rec which contains the information for the opponent as well, we use the following
def opponent_column(stat):
    col=dict()
    for i in range(len(ufcfightscrap['fighter'])):
        if i%2==0:
            col[i]=ufcfightscrap[stat][i+1]
        else:
            col[i]=ufcfightscrap[stat][i-1]
    statdict={'stat':col}
    return pd.DataFrame (statdict, columns = ['stat'])

#enter date unabbreviated 'July 4, 2019'
#here the average gives avg per fight. Later in avg_count we change to average per time spent in octagon
def count(stat, guy,inf_abs, total_L5Y_L2Y_avg, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if total_L5Y_L2Y_avg=='total' or total_L5Y_L2Y_avg=='avg':
        good_indices_1=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0]
    elif total_L5Y_L2Y_avg=='L2Y':
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<730]
    else:
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<1825]
    if inf_abs=='inf':
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    else:
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy]
    good_indices=[i for i in good_indices_1 if i in good_indices_2]
    if total_L5Y_L2Y_avg!='avg':
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
    else:
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
        day1=convert_date_to_abbrev(day1)
        number_fights=wins_before(guy,day1)+losses_before(guy,day1)
        summ=summ/float(number_fights)
    return summ
        
#note a better average is per time not per fight. We will do each stat as an average per one minutes

#enter time in the form 'August 24, 2018'
def time_in_octagon(guy,day1=date.today().strftime('%B %d, %Y')):
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['fighter'][i]==guy] 
    for i in good_indices:
        if ufcfightscrap['time'][i][2]==':':
            summ=int(ufcfightscrap['time'][i][0:2])+int(ufcfightscrap['time'][i][3:])/60.0
        else:
            summ+=5*(ufcfightscrap['round'][i]-1)+int(ufcfightscrap['time'][i][0])+int(ufcfightscrap['time'][i][2:])/60.0
    return summ

#enter date unabbreviated 'July 4, 2019'
#gives takedowns per minute
def avg_count(stat, guy,inf_abs, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if inf_abs=='inf':
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    else:
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    for i in good_indices:
        summ+=ufcfightscrap[stat][i]
    t= time_in_octagon(guy,day1)
    if t==0:
        summ=0
    else:
        summ=summ/t
    return summ

#vectorize these functions
count_vect= npy.vectorize(count)
avg_count_vect= npy.vectorize(avg_count)

#need to define a model first before calling the following functions
#REDEFINE THIS AFTER GETTING ALL STATS COMPILED. WE CURRENTLY CANNOT PREDICT
#I've redefined this in such a way to predict what happens when fighter1 in their day1 version fights fighter2
#in their day2 version. Meaning we could compare for example 2014 Tyron Woodley to 2019 Colby Covington
def ufc_prediction_tuple(fighter1,fighter2,day1=date.today(),day2=date.today()):
    return [wins_before(fighter1,day1)-wins_before(fighter2,day2),
            losses_before(fighter1,day1)-losses_before(fighter2,day2),
            L5Y_wins(fighter1,day1)-L5Y_wins(fighter2,day2),
            L5Y_losses(fighter1,day1)-L5Y_losses(fighter2,day2),
            L2Y_wins(fighter1,day1)-L2Y_wins(fighter2,day2),
            L2Y_losses(fighter1,day1)-L2Y_losses(fighter2,day2),
            ko_wins(fighter1,day1)-ko_wins(fighter2,day2),
            ko_losses(fighter1,day1)-ko_losses(fighter2,day2),
            L5Y_ko_wins(fighter1,day1)-L5Y_ko_wins(fighter2,day2),
            L5Y_ko_losses(fighter1,day1)-L5Y_ko_losses(fighter2,day2),
            L2Y_ko_wins(fighter1,day1)-L2Y_ko_wins(fighter2,day2),
            L2Y_ko_losses(fighter1,day1)-L2Y_ko_losses(fighter2,day2),
            sub_wins(fighter1,day1)-sub_wins(fighter2,day2),
            sub_losses(fighter1,day1)-sub_losses(fighter2,day2),
            L5Y_sub_wins(fighter1,day1)-L5Y_sub_wins(fighter2,day2),
            L5Y_sub_losses(fighter1,day1)-L5Y_sub_losses(fighter2,day2),
            L2Y_sub_wins(fighter1,day1)-L2Y_sub_wins(fighter2,day2),
            L2Y_sub_losses(fighter1,day1)-L2Y_sub_losses(fighter2,day2),
            fighter_age(fighter1,day1),fighter_age(fighter2,day2),
            fighter_height(fighter1)-fighter_height(fighter2),
            fighter_reach(fighter1)-fighter_reach(fighter2),
            avg_count('knockdowns',fighter1,'inf',day1), avg_count('pass',fighter1,'inf',day1),
            avg_count('reversals',fighter1,'inf',day1),avg_count('sub_attempts',fighter1,'inf',day1),
            avg_count('takedowns_landed',fighter1,'inf',day1),avg_count('takedowns_attempts',fighter1,'inf',day1),
            avg_count('sig_strikes_landed',fighter1,'inf',day1),avg_count('sig_strikes_attempts',fighter1,'inf',day1),
            avg_count('total_strikes_landed',fighter1,'inf',day1),avg_count('total_strikes_attempts',fighter1,'inf',day1),
            avg_count('head_strikes_landed',fighter1,'inf',day1),avg_count('head_strikes_attempts',fighter1,'inf',day1),
            avg_count('body_strikes_landed',fighter1,'inf',day1),avg_count('body_strikes_attempts',fighter1,'inf',day1),
            avg_count('leg_strikes_landed',fighter1,'inf',day1),avg_count('leg_strikes_attempts',fighter1,'inf',day1),
            avg_count('distance_strikes_landed',fighter1,'inf',day1),avg_count('distance_strikes_attempts',fighter1,'inf',day1),
            avg_count('clinch_strikes_landed',fighter1,'inf',day1),avg_count('clinch_strikes_attempts',fighter1,'inf',day1),
            avg_count('ground_strikes_landed',fighter1,'inf',day1),avg_count('ground_strikes_attempts',fighter1,'inf',day1),
            avg_count('knockdowns',fighter1,'abs',day1), avg_count('pass',fighter1,'abs',day1),
            avg_count('reversals',fighter1,'abs',day1),avg_count('sub_attempts',fighter1,'abs',day1),
            avg_count('takedowns_landed',fighter1,'abs',day1),avg_count('takedowns_attempts',fighter1,'abs',day1),
            avg_count('sig_strikes_landed',fighter1,'abs',day1),avg_count('sig_strikes_attempts',fighter1,'abs',day1),
            avg_count('total_strikes_landed',fighter1,'abs',day1),avg_count('total_strikes_attempts',fighter1,'abs',day1),
            avg_count('head_strikes_landed',fighter1,'abs',day1),avg_count('head_strikes_attempts',fighter1,'abs',day1),
            avg_count('body_strikes_landed',fighter1,'abs',day1),avg_count('body_strikes_attempts',fighter1,'abs',day1),
            avg_count('leg_strikes_landed',fighter1,'abs',day1),avg_count('leg_strikes_attempts',fighter1,'abs',day1),
            avg_count('distance_strikes_landed',fighter1,'abs',day1),avg_count('distance_strikes_attempts',fighter1,'abs',day1),
            avg_count('clinch_strikes_landed',fighter1,'abs',day1),avg_count('clinch_strikes_attempts',fighter1,'abs',day1),
            avg_count('ground_strikes_landed',fighter1,'abs',day1),avg_count('ground_strikes_attempts',fighter1,'abs',day1),
            avg_count('knockdowns',fighter2,'inf',day2), avg_count('pass',fighter2,'inf',day2),
            avg_count('reversals',fighter2,'inf',day2),avg_count('sub_attempts',fighter2,'inf',day2),
            avg_count('takedowns_landed',fighter2,'inf',day2),avg_count('takedowns_attempts',fighter2,'inf',day2),
            avg_count('sig_strikes_landed',fighter2,'inf',day2),avg_count('sig_strikes_attempts',fighter2,'inf',day2),
            avg_count('total_strikes_landed',fighter2,'inf',day2),avg_count('total_strikes_attempts',fighter2,'inf',day2),
            avg_count('head_strikes_landed',fighter2,'inf',day2),avg_count('head_strikes_attempts',fighter2,'inf',day2),
            avg_count('body_strikes_landed',fighter2,'inf',day2),avg_count('body_strikes_attempts',fighter2,'inf',day2),
            avg_count('leg_strikes_landed',fighter2,'inf',day2),avg_count('leg_strikes_attempts',fighter2,'inf',day2),
            avg_count('distance_strikes_landed',fighter2,'inf',day2),avg_count('distance_strikes_attempts',fighter2,'inf',day2),
            avg_count('clinch_strikes_landed',fighter2,'inf',day2),avg_count('clinch_strikes_attempts',fighter2,'inf',day2),
            avg_count('ground_strikes_landed',fighter2,'inf',day2),avg_count('ground_strikes_attempts',fighter2,'inf',day2),
            avg_count('knockdowns',fighter2,'abs',day2), avg_count('pass',fighter2,'abs',day2),
            avg_count('reversals',fighter2,'abs',day2),avg_count('sub_attempts',fighter2,'abs',day2),
            avg_count('takedowns_landed',fighter2,'abs',day2),avg_count('takedowns_attempts',fighter2,'abs',day2),
            avg_count('sig_strikes_landed',fighter2,'abs',day2),avg_count('sig_strikes_attempts',fighter2,'abs',day2),
            avg_count('total_strikes_landed',fighter2,'abs',day2),avg_count('total_strikes_attempts',fighter2,'abs',day2),
            avg_count('head_strikes_landed',fighter2,'abs',day2),avg_count('head_strikes_attempts',fighter2,'abs',day2),
            avg_count('body_strikes_landed',fighter2,'abs',day2),avg_count('body_strikes_attempts',fighter2,'abs',day2),
            avg_count('leg_strikes_landed',fighter2,'abs',day2),avg_count('leg_strikes_attempts',fighter2,'abs',day2),
            avg_count('distance_strikes_landed',fighter2,'abs',day2),avg_count('distance_strikes_attempts',fighter2,'abs',day2),
            avg_count('clinch_strikes_landed',fighter2,'abs',day2),avg_count('clinch_strikes_attempts',fighter2,'abs',day2),
            avg_count('ground_strikes_landed',fighter2,'abs',day2),avg_count('ground_strikes_attempts',fighter2,'abs',day2),
            stance(fighter1),stance(fighter2)]


#date entered in the form 'Aug 20, 2013'
#predicts what happens if the day1 version of fighter1 fights fighter2 at their day2 version
#by default day1 and day2 are set to today if left unspecified
def ufc_predict(fighter1,fighter2,day1=date.today(), day2=date.today()):
    fighters=[fighter1,fighter2]
    random_guy=fighters[random.choice([0,1])]
    fighter1_fights=(ufcfightscrap['fighter']==fighter1).sum()
    fighter2_fights=(ufcfightscrap['fighter']==fighter2).sum()
    if fighter1_fights==0 and not fighter2_fights==0:
        print('Either you spelled something wrong, or this is '+fighter1+"'s UFC debut... so obviously I do not have enough data to make an educated pick. But since you are DEMANDING an outcome, hmmm... I guess I'll go with "+random_guy+'.')
    elif fighter2_fights==0 and not fighter1_fights==0:
        print('Either you spelled something wrong, or this is '+fighter2+"'s UFC debut... so obviously I do not have enough data to make an educated pick. But since you are DEMANDING an outcome, hmmm... I guess I'll go with "+random_guy+'.') 
    elif fighter1_fights==0 and fighter2_fights==0:
        print('Either you spelled some things wrong, or this is both '+fighter1+"'s "+'and '+fighter2+"'s "+"UFC debuts... so obviously I do not have enough data to make an educated pick. But since you are DEMANDING an outcome, hmmm... I guess I'll go with "+random_guy+'.')
    else:
        ufc_tuple=ufc_prediction_tuple(fighter1,fighter2,day1,day2)
        if 'unknown' in ufc_tuple or 'NaN' in ufc_tuple:
            print('Fighters have incomplete data on record, I honestly do not know what to do with this.')
        else:
            value=model.predict([ufc_tuple])[0]
            result=mlp.predict([ufc_tuple])[0]
            if value=='W':
                print(fighter1+' wins by '+result)
            else:
                print(fighter2+' wins by '+result)

def stance(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['stance'][i]
            break
    if a=='Orthodox':
        return 0
    elif a=='Switch':
        return 1
    elif a=='Southpaw':
        return 2
    elif a=='Open Stance':
        return 3
    elif a=='Sideways':
        return 4
    else:
        return 5
    
stance_vect= npy.vectorize(stance)

def subsets(n):
    lists=[]
    for i in range(1,n+1):
        lists.append(i)
    subsets=[[]]
    n=len(lists)
    for i in range(1,n+1):
        data = itertools.combinations(lists,i)
        subsets.extend(list(data))
    return subsets

In [3]:
ufc_fights= pd.read_csv('ufc_fights.csv')
ufc_fights_crap= pd.read_csv('ufc_fights_crap.csv')
ufcfightscrap= pd.read_csv('ufc_fights_crap.csv')
ufcfighterscrap=pd.read_csv('fighter_stats.csv',sep=',')

/opt/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (9,73) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (36,37,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#adding mask to get rid of unknown stance
stance_mask_a=ufc_fights['fighter_stance']!=5
stance_mask_b=ufc_fights['opponent_stance']!=5
ufc_fights_stance=ufc_fights[stance_mask_a&stance_mask_b]

#gets rid of all unknown, NaN in ufc_fights_stance
mask=ufc_fights_stance['fighter_wins'] != 'unknown'

for col in ufc_fights_stance.columns:
    mask1=ufc_fights_stance[col] != 'unknown'
    mask2=ufc_fights_stance[col] != 'NaN'
    mask3=ufc_fights_stance[col] != '--'
    mask=mask&mask1&mask2&mask3

#gets rid of ufc debut in ufc_fights_stance
maska=ufc_fights_stance['fighter_inf_sig_strikes_attempts_avg'] != 0
maskb=ufc_fights_stance['opponent_inf_sig_strikes_attempts_avg'] != 0
mask=mask&maska&maskb

#applies all masks resulting in ufc_fights_clean_stance (annoying notation... clean the whole thing up)
ufc_fights_clean_stance=ufc_fights_stance[mask]
results=ufc_fights_clean_stance['result']
ufc_fights_ML=ufc_fights_clean_stance.drop(['result','fighter','opponent','method','date','division'], axis=1)

record_statistics=[u'fighter_wins', 
                   u'fighter_losses',
                   u'fighter_L5Y_wins',
                   u'fighter_L5Y_losses', 
                   u'fighter_L2Y_wins', 
                   u'fighter_L2Y_losses',
                    u'fighter_ko_wins',
                   u'fighter_ko_losses',
                   u'fighter_L5Y_ko_wins',
                   u'fighter_L5Y_ko_losses',
                   u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',
                   u'fighter_sub_wins',
                   u'fighter_sub_losses',
                   u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', 
                   u'fighter_L2Y_sub_wins', 
                   u'fighter_L2Y_sub_losses',
                   u'opponent_wins', 
                   u'opponent_losses',
                   u'opponent_L5Y_wins', 
                   u'opponent_L5Y_losses', 
                   u'opponent_L2Y_wins', 
                   u'opponent_L2Y_losses', 
                    u'opponent_ko_wins', 
                   u'opponent_ko_losses', 
                   u'opponent_L5Y_ko_wins', 
                   u'opponent_L5Y_ko_losses', 
                   u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses', 
                   u'opponent_sub_wins', 
                   u'opponent_sub_losses',
                   u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', 
                   u'opponent_L2Y_sub_wins', 
                   u'opponent_L2Y_sub_losses']

physical_stats=[ u'fighter_age',
                u'fighter_height',
                    u'fighter_reach',  
                u'opponent_age',  
                u'opponent_height',
                    u'opponent_reach']

#here is the list of all stats available (besides stance), does not include names or result
punch_statistics=[    u'fighter_inf_knockdowns_avg',
                    u'fighter_inf_pass_avg',
                    u'fighter_inf_reversals_avg',
                    u'fighter_inf_sub_attempts_avg',
                    u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                
                    u'fighter_abs_knockdowns_avg',
                    u'fighter_abs_pass_avg',
                    u'fighter_abs_reversals_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                
                    u'opponent_abs_knockdowns_avg',
                    u'opponent_abs_pass_avg',
                    u'opponent_abs_reversals_avg',
                    u'opponent_abs_sub_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg']

#forming a dataframe where columns are differences between fighter and opponent fight statistics
mrs=len(record_statistics)/2
record_diff_2_df=pd.DataFrame()
for i in range(mrs):
    record_diff_2_df[record_statistics[i]+'_diff_2']=ufc_fights_clean_stance[record_statistics[i]]-ufc_fights_clean_stance[record_statistics[i+mrs]]
    
#Lets start building a dataframe of all the stats which we know perform best
ufc_best_df=record_diff_2_df

#moving on to physical_stats:
physical_stats_diff_df=pd.DataFrame()
physical=[ [u'fighter_age',u'opponent_age'],[u'fighter_height',u'opponent_height'],[u'fighter_reach',u'opponent_reach']]
physical_stats_diff_df['age_diff']=ufc_fights_clean_stance['fighter_age'].apply(float)-ufc_fights_clean_stance['opponent_age'].apply(float)
physical_stats_diff_df['height_diff']=ufc_fights_clean_stance['fighter_height'].apply(float)-ufc_fights_clean_stance['opponent_height'].apply(float)
physical_stats_diff_df['reach_diff']=ufc_fights_clean_stance['fighter_reach'].apply(float)-ufc_fights_clean_stance['opponent_reach'].apply(float)
#ok so we keep ages absolute, but make differences for height and reach
ufc_best_df['fighter_age']=ufc_fights_clean_stance['fighter_age']
ufc_best_df['opponent_age']=ufc_fights_clean_stance['opponent_age']
ufc_best_df['height_diff']=physical_stats_diff_df['height_diff']
ufc_best_df['reach_diff']=physical_stats_diff_df['reach_diff']
#This includes all the best choices in terms of taking differences
for col in punch_statistics:
    ufc_best_df[col]=ufc_fights_clean_stance[col]

/opt/anaconda2/lib/python2.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [6]:
#Heres the score for our "best choices" (without stance included)
#THIS IS THE HIGHEST AVERAGE WE HAVE EVER GOTTEN, ITS A CROSS_VAL_SCORE of .71, (changed since then...)
#increases with cv (testing optimal value)
X=ufc_best_df.iloc[0:2101]
y=ufc_fights_clean_stance.iloc[0:2101]['result']
cross_val_score(model,X,y,cv=23).mean()

/opt/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7036075191755417

In [7]:
#adding stance to ufc_best_df
ufc_best_df['fighter_stance']=ufc_fights_clean_stance['fighter_stance']
ufc_best_df['opponent_stance']=ufc_fights_clean_stance['opponent_stance']

In [8]:
#Heres the score for our "best choices" (WITH stance included, increased by .005)
#THIS IS THE HIGHEST AVERAGE WE HAVE EVER GOTTEN, ITS A CROSS_VAL_SCORE of .71, (changed after update...)
#increases with cv (testing optimal value)
X=ufc_best_df.iloc[0:2101]
y=ufc_fights_clean_stance.iloc[0:2101]['result']
cross_val_score(model,X,y,cv=23).mean()

0.7040796510642232

In [12]:
#Getting rid of all fights whose method is DQ, Overturned, or CNC or any draws
method_mask1=ufc_fights_clean_stance['method']!='DQ'
method_mask2=ufc_fights_clean_stance['method']!='Overturned'
method_mask3=ufc_fights_clean_stance['method']!='CNC'
method_mask4=ufc_fights_clean_stance['result']!='D'
method_mask=method_mask1&method_mask2&method_mask3&method_mask4
method_mask.sum()

3620

In [13]:
#by removing all the bullshit I'm able to increase accuracy by almost .01
model=LogisticRegression()
XX=ufc_best_df[method_mask].iloc[0:2101]
yy=ufc_fights_clean_stance[method_mask].iloc[0:2101]['result']
cross_val_score(model,XX,yy,cv=23).mean()

0.7124777218837832

In [14]:
#Now lets get rid of the different types of decisions (all become DEC)
ufc_fights_clean_stance['method'] = ufc_fights_clean_stance['method'].replace(['U-DEC'],'DEC')
ufc_fights_clean_stance['method'] = ufc_fights_clean_stance['method'].replace(['S-DEC'],'DEC')
ufc_fights_clean_stance['method'] = ufc_fights_clean_stance['method'].replace(['M-DEC'],'DEC')

/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [15]:
#we're up to 50% for method predictions:
mlp=MLPClassifier(max_iter=1000,hidden_layer_sizes=(110,110),alpha=.00001,solver='sgd',random_state=3)
Xp=ufc_best_df[method_mask].iloc[0:2101]
yp=ufc_fights_clean_stance[method_mask].iloc[0:2101]['method']
cross_val_score(model,Xp,yp,cv=10).mean()

0.5073958716721875

In [11]:
#setting the currest strongest models for result and method prediction: model and mlp respectively
model=LogisticRegression()
XX=ufc_best_df[method_mask].iloc[0:2101]
yy=ufc_fights_clean_stance[method_mask].iloc[0:2101]['result']
model.fit(XX,yy)
model.score(XX,yy)

0.7496430271299381

In [12]:
#this score is of course way overfit (its a neural network)
mlp=MLPClassifier(max_iter=1000,hidden_layer_sizes=(110,110),alpha=.00001,solver='sgd',random_state=3)
Xp=ufc_best_df[method_mask].iloc[0:2101]
yp=ufc_fights_clean_stance[method_mask].iloc[0:2101]['method']
mlp.fit(Xp,yp)
mlp.score(Xp,yp)

0.8653022370299858

In [ ]:
#The following are examples of some recent fights. You don't have to run these cells (but run everything above
#this cell in order)

In [18]:
ufc_predict('Augusto Sakai','Alistair Overeem','September 6, 2020', 'September 6, 2020')

Augusto Sakai wins by DEC


In [20]:
for col in ufc_best_df.columns:
    print("'"+col+"',")

'fighter_wins_diff_2',
'fighter_losses_diff_2',
'fighter_L5Y_wins_diff_2',
'fighter_L5Y_losses_diff_2',
'fighter_L2Y_wins_diff_2',
'fighter_L2Y_losses_diff_2',
'fighter_ko_wins_diff_2',
'fighter_ko_losses_diff_2',
'fighter_L5Y_ko_wins_diff_2',
'fighter_L5Y_ko_losses_diff_2',
'fighter_L2Y_ko_wins_diff_2',
'fighter_L2Y_ko_losses_diff_2',
'fighter_sub_wins_diff_2',
'fighter_sub_losses_diff_2',
'fighter_L5Y_sub_wins_diff_2',
'fighter_L5Y_sub_losses_diff_2',
'fighter_L2Y_sub_wins_diff_2',
'fighter_L2Y_sub_losses_diff_2',
'fighter_age',
'opponent_age',
'height_diff',
'reach_diff',
'fighter_inf_knockdowns_avg',
'fighter_inf_pass_avg',
'fighter_inf_reversals_avg',
'fighter_inf_sub_attempts_avg',
'fighter_inf_takedowns_landed_avg',
'fighter_inf_takedowns_attempts_avg',
'fighter_inf_sig_strikes_landed_avg',
'fighter_inf_sig_strikes_attempts_avg',
'fighter_inf_total_strikes_landed_avg',
'fighter_inf_total_strikes_attempts_avg',
'fighter_inf_head_strikes_landed_avg',
'fighter_inf_head_strikes_atte

In [26]:
rf=RandomForestClassifier()
X=ufc_best_df.iloc[0:2101]
y=ufc_fights_clean_stance.iloc[0:2101]['result']
rf.fit(X,y)
ft_impo=pd.Series(rf.feature_importances_,index=ufc_best_df.columns).sort_values(ascending=False)
ft_impo.head(21)

fighter_ko_wins_diff_2                        0.028724
fighter_age                                   0.017369
fighter_sub_wins_diff_2                       0.016273
fighter_abs_total_strikes_landed_avg          0.014774
opponent_abs_clinch_strikes_attempts_avg      0.014265
fighter_abs_clinch_strikes_landed_avg         0.013957
opponent_age                                  0.013875
opponent_abs_total_strikes_landed_avg         0.013786
opponent_inf_sig_strikes_landed_avg           0.013193
fighter_inf_takedowns_attempts_avg            0.012526
opponent_inf_distance_strikes_attempts_avg    0.012457
opponent_abs_ground_strikes_attempts_avg      0.012368
fighter_abs_total_strikes_attempts_avg        0.012085
opponent_abs_sig_strikes_landed_avg           0.012050
fighter_abs_sig_strikes_landed_avg            0.011963
opponent_abs_sig_strikes_attempts_avg         0.011717
opponent_inf_body_strikes_landed_avg          0.011536
fighter_abs_head_strikes_landed_avg           0.011490
opponent_i

In [16]:
keepers=['fighter_ko_wins_diff_2',
        'fighter_inf_total_strikes_landed_avg',
        'opponent_inf_total_strikes_landed_avg',
        'fighter_abs_total_strikes_landed_avg',
        'opponent_abs_total_strikes_landed_avg',
        'fighter_age',
        'opponent_age',
        'fighter_sub_wins_diff_2',
        'fighter_inf_head_strikes_landed_avg',
        'fighter_abs_head_strikes_landed_avg',
        'opponent_inf_head_strikes_landed_avg',
        'opponent_abs_head_strikes_landed_avg',
        'fighter_ko_losses_diff_2',
        'fighter_inf_ground_strikes_landed_avg',
        'fighter_abs_ground_strikes_landed_avg',
        'opponent_inf_ground_strikes_landed_avg',
        'opponent_abs_ground_strikes_landed_avg',
        'fighter_inf_body_strikes_landed_avg',
        'fighter_inf_body_strikes_attempts_avg',
        'fighter_abs_body_strikes_landed_avg',
        'fighter_abs_body_strikes_attempts_avg',
        'opponent_inf_body_strikes_landed_avg',
        'opponent_inf_body_strikes_attempts_avg',
        'opponent_abs_body_strikes_landed_avg',
        'opponent_abs_body_strikes_attempts_avg',
        'fighter_inf_takedowns_landed_avg',
        'fighter_abs_takedowns_landed_avg',
        'opponent_inf_takedowns_landed_avg',
        'opponent_abs_takedowns_landed_avg',
        'fighter_inf_knockdowns_avg',
        'fighter_abs_knockdowns_avg',
        'opponent_inf_knockdowns_avg',
        'opponent_abs_knockdowns_avg',
        'fighter_inf_ground_strikes_attempts_avg',
        'fighter_abs_ground_strikes_attempts_avg',
        'opponent_inf_ground_strikes_attempts_avg',
        'opponent_abs_ground_strikes_attempts_avg',
        'fighter_inf_leg_strikes_landed_avg',
        'fighter_inf_leg_strikes_attempts_avg',
        'fighter_abs_leg_strikes_landed_avg',
        'fighter_abs_leg_strikes_attempts_avg',
        'opponent_inf_leg_strikes_landed_avg',
        'opponent_inf_leg_strikes_attempts_avg',
        'opponent_abs_leg_strikes_landed_avg',
        'opponent_abs_leg_strikes_attempts_avg',
        'fighter_inf_clinch_strikes_attempts_avg',
        'fighter_abs_clinch_strikes_attempts_avg',
        'opponent_inf_clinch_strikes_attempts_avg',
        'opponent_abs_clinch_strikes_attempts_avg',
        'fighter_inf_pass_avg',
        'fighter_abs_pass_avg',
        'opponent_inf_pass_avg',
        'opponent_abs_pass_avg',
        'fighter_stance',
        'opponent_stance',
        'height_diff',
        'reach_diff']

maybies=[
['fighter_wins_diff_2',
'fighter_losses_diff_2'],
['fighter_L5Y_wins_diff_2',
'fighter_L5Y_losses_diff_2',
'fighter_L2Y_wins_diff_2',
'fighter_L2Y_losses_diff_2'],
['fighter_L5Y_ko_wins_diff_2',
'fighter_L5Y_ko_losses_diff_2',
'fighter_L2Y_ko_wins_diff_2',
'fighter_L2Y_ko_losses_diff_2'],
['fighter_sub_losses_diff_2',
'fighter_L5Y_sub_wins_diff_2',
'fighter_L5Y_sub_losses_diff_2',
'fighter_L2Y_sub_wins_diff_2',
'fighter_L2Y_sub_losses_diff_2'],
['fighter_inf_reversals_avg',
'fighter_abs_reversals_avg',
'opponent_inf_reversals_avg',
'opponent_abs_reversals_avg'],
['fighter_inf_sub_attempts_avg',
'fighter_abs_sub_attempts_avg',
'opponent_inf_sub_attempts_avg',
'opponent_abs_sub_attempts_avg'],
['fighter_inf_takedowns_attempts_avg',
'fighter_abs_takedowns_attempts_avg',
'opponent_inf_takedowns_attempts_avg',
'opponent_abs_takedowns_attempts_avg'],
['fighter_inf_sig_strikes_landed_avg',
'fighter_abs_sig_strikes_landed_avg',
'opponent_inf_sig_strikes_landed_avg',
'opponent_abs_sig_strikes_landed_avg'],
['fighter_inf_sig_strikes_attempts_avg',
'fighter_abs_sig_strikes_attempts_avg',
'opponent_inf_sig_strikes_attempts_avg',
'opponent_abs_sig_strikes_attempts_avg'],
['fighter_inf_total_strikes_attempts_avg',
'fighter_abs_total_strikes_attempts_avg',
'opponent_inf_total_strikes_attempts_avg',
'opponent_abs_total_strikes_attempts_avg'],
['fighter_inf_head_strikes_attempts_avg',
'fighter_abs_head_strikes_attempts_avg',
'opponent_inf_head_strikes_attempts_avg',
'opponent_abs_head_strikes_attempts_avg'],
['fighter_inf_distance_strikes_landed_avg',
'fighter_abs_distance_strikes_landed_avg',
'opponent_inf_distance_strikes_landed_avg',
'opponent_abs_distance_strikes_landed_avg'],
['fighter_inf_distance_strikes_attempts_avg',
'fighter_abs_distance_strikes_attempts_avg',
'opponent_inf_distance_strikes_attempts_avg',
'opponent_abs_distance_strikes_attempts_avg'],
['fighter_inf_clinch_strikes_landed_avg',
'fighter_abs_clinch_strikes_landed_avg',
'opponent_inf_clinch_strikes_landed_avg',
'opponent_abs_clinch_strikes_landed_avg'],
]
len(maybies)

14

In [19]:
#Lets find the best model we can. We'll start with some "keepers" and add in the less desirable stats 
#Consider removing stats like 'absorbed leg strike attempts'... honestly how much info could this capture

In [32]:
#this is the time in hours it should take to complete this computation
len(subsets(14))*2/60/60

9

In [55]:
#keeping all the keepers for now and seeing which subset of maybies to add
data_points=[]
SUB=subsets(14)
for S in SUB:
    candidate=[]
    candidate.extend(keepers)
    for i in S:
        candidate.extend(maybies[i-1])
    model=LogisticRegression()
    XXX=ufc_best_df[method_mask][candidate].iloc[0:2101]
    yyy=ufc_fights_clean_stance[method_mask].iloc[0:2101]['result']
    point=[S,cross_val_score(model,XXX,yyy,cv=5).mean()]
    data_points.append(point)
    print(point)

[[], 0.6463691890057686]
[(1,), 0.6582671643479243]
[(2,), 0.6501821060965954]
[(3,), 0.6516038909625608]
[(4,), 0.6568329374505145]
[(5,), 0.6444655581947742]
[(6,), 0.6444689514760773]
[(7,), 0.6520778192512159]
[(8,), 0.6463703200995361]
[(9,), 0.6411367492365116]
[(10,), 0.6387569279493269]
[(11,), 0.6411367492365116]
[(12,), 0.6439904988123515]
[(13,), 0.6416129397127022]
[(14,), 0.6430369867662028]
[(1, 2), 0.6639803189684425]
[(1, 3), 0.6692127587376994]
[(1, 4), 0.6687320438864381]
[(1, 5), 0.6573170455830788]
[(1, 6), 0.6596991290577989]
[(1, 7), 0.6639825811559777]
[(1, 8), 0.6563646646306978]
[(1, 9), 0.6577887116841985]
[(1, 10), 0.6582671643479243]
[(1, 11), 0.6573136523017757]
[(1, 12), 0.6596957357764959]
[(1, 13), 0.6577887116841985]
[(1, 14), 0.6587444859178826]
[(2, 3), 0.6592206763940731]
[(2, 4), 0.6549293066395203]
[(2, 5), 0.6501821060965954]
[(2, 6), 0.6516118086189345]
[(2, 7), 0.6520879990951249]
[(2, 8), 0.6492308562379822]
[(2, 9), 0.6401843682841307]
[(2, 10

[(3, 5, 8), 0.6496991290577989]
[(3, 5, 9), 0.6477966293405724]
[(3, 5, 10), 0.6511277004863703]
[(3, 5, 11), 0.6477954982468046]
[(3, 5, 12), 0.6506526411039475]
[(3, 5, 13), 0.6492252007691437]
[(3, 5, 14), 0.6487501413867209]
[(3, 6, 7), 0.6625438298834972]
[(3, 6, 8), 0.6554111525845492]
[(3, 6, 9), 0.6487456170116502]
[(3, 6, 10), 0.6492218074878406]
[(3, 6, 11), 0.6496979979640312]
[(3, 6, 12), 0.6530335934848999]
[(3, 6, 13), 0.6492218074878406]
[(3, 6, 14), 0.6492263318629113]
[(3, 7, 8), 0.6611197828299965]
[(3, 7, 9), 0.6601651396900803]
[(3, 7, 10), 0.6568351996380499]
[(3, 7, 11), 0.6596889492138898]
[(3, 7, 12), 0.6596946046827281]
[(3, 7, 13), 0.6592138898314669]
[(3, 7, 14), 0.6592161520190024]
[(3, 8, 9), 0.6506526411039475]
[(3, 8, 10), 0.6511277004863703]
[(3, 8, 11), 0.650176450627757]
[(3, 8, 12), 0.6473193077706141]
[(3, 8, 13), 0.6497002601515665]
[(3, 8, 14), 0.6477988915281077]
[(3, 9, 10), 0.6506503789164121]
[(3, 9, 11), 0.647794367153037]
[(3, 9, 12), 0.64874

[(1, 2, 6, 14), 0.6654088903970139]
[(1, 2, 7, 8), 0.6730222825472232]
[(1, 2, 7, 9), 0.6668340685442823]
[(1, 2, 7, 10), 0.6663578780680919]
[(1, 2, 7, 11), 0.6673102590204728]
[(1, 2, 7, 12), 0.6720732948761453]
[(1, 2, 7, 13), 0.6668340685442823]
[(1, 2, 7, 14), 0.6706447234475739]
[(1, 2, 8, 9), 0.660169664065151]
[(1, 2, 8, 10), 0.661595973306187]
[(1, 2, 8, 11), 0.6596934735889605]
[(1, 2, 8, 12), 0.6654066282094786]
[(1, 2, 8, 13), 0.6601696640651509]
[(1, 2, 8, 14), 0.667309127926705]
[(1, 2, 9, 10), 0.6620710326886099]
[(1, 2, 9, 11), 0.661595973306187]
[(1, 2, 9, 12), 0.6592172831127701]
[(1, 2, 9, 13), 0.6611209139237643]
[(1, 2, 9, 14), 0.6615937111186517]
[(1, 2, 10, 11), 0.6634996041171812]
[(1, 2, 10, 12), 0.6601685329713833]
[(1, 2, 10, 13), 0.6620710326886099]
[(1, 2, 10, 14), 0.6611209139237643]
[(1, 2, 11, 12), 0.6596923424951928]
[(1, 2, 11, 13), 0.6611197828299965]
[(1, 2, 11, 14), 0.6611175206424613]
[(1, 2, 12, 13), 0.6587410926365795]
[(1, 2, 12, 14), 0.66731025

[(1, 8, 13, 14), 0.6587433548241148]
[(1, 9, 10, 11), 0.657794367153037]
[(1, 9, 10, 12), 0.6592229385816084]
[(1, 9, 10, 13), 0.657794367153037]
[(1, 9, 10, 14), 0.6596968668702635]
[(1, 9, 11, 12), 0.6577921049655016]
[(1, 9, 11, 13), 0.6577887116841985]
[(1, 9, 11, 14), 0.6573147833955434]
[(1, 9, 12, 13), 0.6568408551068883]
[(1, 9, 12, 14), 0.659220676394073]
[(1, 9, 13, 14), 0.6577898427779663]
[(1, 10, 11, 12), 0.6601753195339893]
[(1, 10, 11, 13), 0.657794367153037]
[(1, 10, 11, 14), 0.6601730573464539]
[(1, 10, 12, 13), 0.6587478791991855]
[(1, 10, 12, 14), 0.6592229385816084]
[(1, 10, 13, 14), 0.658271688722995]
[(1, 11, 12, 13), 0.6573170455830788]
[(1, 11, 12, 14), 0.659220676394073]
[(1, 11, 13, 14), 0.6577909738717339]
[(1, 12, 13, 14), 0.659220676394073]
[(2, 3, 4, 5), 0.6692082343626287]
[(2, 3, 4, 6), 0.671585793462278]
[(2, 3, 4, 7), 0.6696866870263544]
[(2, 3, 4, 8), 0.6701594842212419]
[(2, 3, 4, 9), 0.6630177581721525]
[(2, 3, 4, 10), 0.6634950797421106]
[(2, 3, 4,

[(2, 9, 11, 14), 0.637807940278249]
[(2, 9, 12, 13), 0.6406616898540889]
[(2, 9, 12, 14), 0.6382829996606718]
[(2, 9, 13, 14), 0.6397127021830109]
[(2, 10, 11, 12), 0.6411378803302793]
[(2, 10, 11, 13), 0.6406594276665535]
[(2, 10, 11, 14), 0.6406605587603212]
[(2, 10, 12, 13), 0.6420891301888927]
[(2, 10, 12, 14), 0.6401877615654337]
[(2, 10, 13, 14), 0.6416129397127022]
[(2, 11, 12, 13), 0.6392331184255174]
[(2, 11, 12, 14), 0.6373306187082909]
[(2, 11, 13, 14), 0.6392353806130527]
[(2, 12, 13, 14), 0.6373317498020585]
[(3, 4, 5, 6), 0.6696832937450515]
[(3, 4, 5, 7), 0.6649179957018437]
[(3, 4, 5, 8), 0.6658749010292954]
[(3, 4, 5, 9), 0.661589186743581]
[(3, 4, 5, 10), 0.6582547223164801]
[(3, 4, 5, 11), 0.661589186743581]
[(3, 4, 5, 12), 0.6682581155977829]
[(3, 4, 5, 13), 0.661589186743581]
[(3, 4, 5, 14), 0.6668261508879085]
[(3, 4, 6, 7), 0.6711062097047844]
[(3, 4, 6, 8), 0.6677796629340572]
[(3, 4, 6, 9), 0.6587343060739734]
[(3, 4, 6, 10), 0.6620665083135392]
[(3, 4, 6, 11),

...

[(2, 3, 5, 6, 7, 9, 11, 12, 13, 14), 0.6577864494966632]
[(2, 3, 5, 6, 7, 10, 11, 12, 13, 14), 0.6563556158805565]
[(2, 3, 5, 6, 8, 9, 10, 11, 12, 13), 0.6554066282094786]
[(2, 3, 5, 6, 8, 9, 10, 11, 12, 14), 0.6558839497794368]
[(2, 3, 5, 6, 8, 9, 10, 11, 13, 14), 0.6563590091618595]
[(2, 3, 5, 6, 8, 9, 10, 12, 13, 14), 0.6582615088790861]
[(2, 3, 5, 6, 8, 9, 11, 12, 13, 14), 0.6539769256871394]
[(2, 3, 5, 6, 8, 10, 11, 12, 13, 14), 0.6563578780680918]
[(2, 3, 5, 6, 9, 10, 11, 12, 13, 14), 0.6563590091618595]
[(2, 3, 5, 7, 8, 9, 10, 11, 12, 13), 0.6539825811559778]
[(2, 3, 5, 7, 8, 9, 10, 11, 12, 14), 0.653504128492252]
[(2, 3, 5, 7, 8, 9, 10, 11, 13, 14), 0.653504128492252]
[(2, 3, 5, 7, 8, 9, 10, 12, 13, 14), 0.652551747539871]
[(2, 3, 5, 7, 8, 9, 11, 12, 13, 14), 0.6539791878746748]
[(2, 3, 5, 7, 8, 10, 11, 12, 13, 14), 0.6535052595860196]
[(2, 3, 5, 7, 9, 10, 11, 12, 13, 14), 0.6516016287750255]
[(2, 3, 5, 8, 9, 10, 11, 12, 13, 14), 0.6554122836783169]
[(2, 3, 6, 7, 8, 9, 10, 11, 

[(4, 5, 7, 8, 9, 10, 11, 12, 13, 14), 0.6539769256871394]
[(4, 6, 7, 8, 9, 10, 11, 12, 13, 14), 0.6530245447347585]
[(5, 6, 7, 8, 9, 10, 11, 12, 13, 14), 0.6463657957244655]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), 0.7125234701956792]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12), 0.7130007917656374]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13), 0.7120484108132564]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14), 0.712524601289447]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12), 0.7115676959619953]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13), 0.7087139463861554]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 14), 0.7115699581495306]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13), 0.7101391245334239]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 14), 0.7106175771971497]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 13, 14), 0.7115699581495306]
[(1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12), 0.7134758511480601]
[(1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13), 0.7134747200542926]
[(1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 14), 0.7130007917656374]
[(1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13), 0.713950910530483]
[(1, 2, 3, 4

[(1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13), 0.6849157335143083]
[(1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 14), 0.6849157335143085]
[(1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14), 0.6849146024205407]
[(1, 2, 3, 5, 6, 7, 9, 10, 12, 13, 14), 0.6844395430381178]
[(1, 2, 3, 5, 6, 7, 9, 11, 12, 13, 14), 0.6877717452776835]
[(1, 2, 3, 5, 6, 7, 10, 11, 12, 13, 14), 0.6839633525619273]
[(1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13), 0.6830121027033143]
[(1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 14), 0.6815835312747428]
[(1, 2, 3, 5, 6, 8, 9, 10, 11, 13, 14), 0.6801549598461714]
[(1, 2, 3, 5, 6, 8, 9, 10, 12, 13, 14), 0.6815846623685105]
[(1, 2, 3, 5, 6, 8, 9, 11, 12, 13, 14), 0.6792071032688611]
[(1, 2, 3, 5, 6, 8, 10, 11, 12, 13, 14), 0.6806334125098971]
[(1, 2, 3, 5, 6, 9, 10, 11, 12, 13, 14), 0.6801572220337067]
[(1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13), 0.6882468046601063]
[(1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 14), 0.686821626512838]
[(1, 2, 3, 5, 7, 8, 9, 10, 11, 13, 14), 0.6863454360366475]
[(1, 2, 3, 5, 7, 8, 9, 10, 12, 13, 14)

[(1, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14), 0.6663601402556272]
[(1, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14), 0.6692116276439318]
[(1, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14), 0.6682603777853184]
[(1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), 0.6744440674131885]
[(1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14), 0.6734962108358783]
[(1, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14), 0.6739712702183012]
[(1, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14), 0.6730200203596878]
[(1, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14), 0.6730177581721524]
[(1, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14), 0.6730200203596878]
[(1, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14), 0.6725438298834974]
[(1, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14), 0.6701651396900803]
[(1, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14), 0.6715914489311163]
[(1, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14), 0.6739712702183012]
[(1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14), 0.6611277004863703]
[(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), 0.6668261508879085]
[(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13), 0.6653941861780341]
[(2, 3, 4, 5, 6, 7, 8, 9, 10, 11,

[(1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 14), 0.7087150774799231]
[(1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14), 0.7115676959619953]
[(1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13), 0.6844406741318856]
[(1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 14), 0.6863443049428799]
[(1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 13, 14), 0.6868204954190704]
[(1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 13, 14), 0.6839633525619273]
[(1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 14), 0.6868193643253026]
[(1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14), 0.6849157335143085]
[(1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14), 0.685391923990499]
[(1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14), 0.680156090939939]
[(1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14), 0.686821626512838]
[(1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14), 0.6849157335143083]
[(1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), 0.6815812690872074]
[(1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14), 0.683483768804434]
[(1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14), 0.6820540662820948]
[(1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14), 0.6806277

In [56]:
M=max([tup[1] for tup in data_points])
M

0.7177593032462392

In [59]:
best_sub=[tupe[0] for tupe in data_points if tupe[1]==M]
best_sub

[(1, 2, 3, 4, 5, 7)]

In [19]:
maybe_list=[]
for i in (1, 2, 3, 4, 5, 7):
    maybe_list.extend(maybies[i-1])
for r in maybe_list:
    print r

fighter_wins_diff_2
fighter_losses_diff_2
fighter_L5Y_wins_diff_2
fighter_L5Y_losses_diff_2
fighter_L2Y_wins_diff_2
fighter_L2Y_losses_diff_2
fighter_L5Y_ko_wins_diff_2
fighter_L5Y_ko_losses_diff_2
fighter_L2Y_ko_wins_diff_2
fighter_L2Y_ko_losses_diff_2
fighter_sub_losses_diff_2
fighter_L5Y_sub_wins_diff_2
fighter_L5Y_sub_losses_diff_2
fighter_L2Y_sub_wins_diff_2
fighter_L2Y_sub_losses_diff_2
fighter_inf_reversals_avg
fighter_abs_reversals_avg
opponent_inf_reversals_avg
opponent_abs_reversals_avg
fighter_inf_takedowns_attempts_avg
fighter_abs_takedowns_attempts_avg
opponent_inf_takedowns_attempts_avg
opponent_abs_takedowns_attempts_avg


In [74]:
winner=[]
winner.extend(keepers)
for i in best_sub[0]:
    winner.extend(maybies[i-1])

In [76]:
model=LogisticRegression()
XXX=ufc_best_df[method_mask][winner].iloc[0:2101]
yyy=ufc_fights_clean_stance[method_mask].iloc[0:2101]['result']
model.fit(XXX,yyy)
model.score(XXX,yyy)

0.7496430271299381

In [78]:
#we have improved a bit. lets do it again
cross_val_score(model,XXX,yyy,cv=23).mean()

0.719624253422046

In [ ]:
#to improve this, we could first OPTIMALLY pick keepers. size of keepers is 57 and size of keepers+maybies is 71
#therefore we must iterate through 71 choose 57 = no... too huge... 10^14
#now we'll loop through subsets of keepers

In [20]:
keeps=[['fighter_ko_wins_diff_2'],
        ['fighter_inf_total_strikes_landed_avg',
        'opponent_inf_total_strikes_landed_avg',
        'fighter_abs_total_strikes_landed_avg',
        'opponent_abs_total_strikes_landed_avg'],
        ['fighter_age',
        'opponent_age'],
        ['fighter_sub_wins_diff_2'],
        ['fighter_inf_head_strikes_landed_avg',
        'fighter_abs_head_strikes_landed_avg',
        'opponent_inf_head_strikes_landed_avg',
        'opponent_abs_head_strikes_landed_avg'],
        ['fighter_ko_losses_diff_2'],
        ['fighter_inf_body_strikes_landed_avg',
        'fighter_inf_body_strikes_attempts_avg',
        'fighter_abs_body_strikes_landed_avg',
        'fighter_abs_body_strikes_attempts_avg',
        'opponent_inf_body_strikes_landed_avg',
        'opponent_inf_body_strikes_attempts_avg',
        'opponent_abs_body_strikes_landed_avg',
        'opponent_abs_body_strikes_attempts_avg'],
        ['fighter_inf_takedowns_landed_avg',
        'fighter_abs_takedowns_landed_avg',
        'opponent_inf_takedowns_landed_avg',
        'opponent_abs_takedowns_landed_avg'],
        ['fighter_inf_knockdowns_avg',
        'fighter_abs_knockdowns_avg',
        'opponent_inf_knockdowns_avg',
        'opponent_abs_knockdowns_avg'],
        ['fighter_inf_ground_strikes_attempts_avg',
        'fighter_abs_ground_strikes_attempts_avg',
        'opponent_inf_ground_strikes_attempts_avg',
        'opponent_abs_ground_strikes_attempts_avg',
        'fighter_inf_ground_strikes_landed_avg',
        'fighter_abs_ground_strikes_landed_avg',
        'opponent_inf_ground_strikes_landed_avg',
        'opponent_abs_ground_strikes_landed_avg'],
        ['fighter_inf_leg_strikes_landed_avg',
        'fighter_inf_leg_strikes_attempts_avg',
        'fighter_abs_leg_strikes_landed_avg',
        'fighter_abs_leg_strikes_attempts_avg',
        'opponent_inf_leg_strikes_landed_avg',
        'opponent_inf_leg_strikes_attempts_avg',
        'opponent_abs_leg_strikes_landed_avg',
        'opponent_abs_leg_strikes_attempts_avg'],
        ['fighter_inf_clinch_strikes_attempts_avg',
        'fighter_abs_clinch_strikes_attempts_avg',
        'opponent_inf_clinch_strikes_attempts_avg',
        'opponent_abs_clinch_strikes_attempts_avg'],
        ['fighter_inf_pass_avg',
        'fighter_abs_pass_avg',
        'opponent_inf_pass_avg',
        'opponent_abs_pass_avg'],
        ['fighter_stance',
        'opponent_stance',
        'height_diff',
        'reach_diff']]
len(keeps)

14

In [102]:
can=[]
for i in best_sub[0]:
    can.extend(maybies[i-1])

In [103]:
can

False

In [104]:
len(can)

23

In [105]:
len(candidate)

135

In [106]:
data_poi=[]
SUB=subsets(14)
for S in SUB:
    candi=[]
    candi.extend(can)
    for i in S:
        candi.extend(keeps[i-1])
    model=LogisticRegression()
    XXXX=ufc_best_df[method_mask][candi].iloc[0:2101]
    yyyy=ufc_fights_clean_stance[method_mask].iloc[0:2101]['result']
    point=[S,cross_val_score(model,XXXX,yyyy,cv=5).mean()]
    data_poi.append(point)
    print(point)

[[], 0.5768759190136862]
[(1,), 0.6896810315575161]
[(2,), 0.5887705010745391]
[(3,), 0.6159009161859518]
[(4,), 0.6111446668928855]
[(5,), 0.5906775251668364]
[(6,), 0.5844757380386835]
[(7,), 0.5825890736342043]
[(8,), 0.5792512159258003]
[(9,), 0.5797330618708292]
[(10,), 0.5778226444972289]
[(11,), 0.5725879425404365]
[(12,), 0.5854416921162764]
[(13,), 0.581162764393168]
[(14,), 0.5835437167741206]
[(1, 2), 0.6882569845040154]
[(1, 3), 0.6972966858952608]
[(1, 4), 0.7210801945481281]
[(1, 5), 0.6968227576066056]
[(1, 6), 0.6777717452776835]
[(1, 7), 0.680169664065151]
[(1, 8), 0.6849213889831468]
[(1, 9), 0.6858715077479923]
[(1, 10), 0.6777762696527543]
[(1, 11), 0.6801594842212421]
[(1, 12), 0.683487162085737]
[(1, 13), 0.6849225200769145]
[(1, 14), 0.6806368057912001]
[(2, 3), 0.624466689288542]
[(2, 4), 0.6178102024657844]
[(2, 5), 0.5887806809184482]
[(2, 6), 0.5935143083361611]
[(2, 7), 0.5873487162085738]
[(2, 8), 0.5901979414093429]
[(2, 9), 0.5925766316027599]
[(2, 10), 0

[(3, 5, 8), 0.6225630584775478]
[(3, 5, 9), 0.6239882366248162]
[(3, 5, 10), 0.6206571654790183]
[(3, 5, 11), 0.61541567695962]
[(3, 5, 12), 0.6192240696753761]
[(3, 5, 13), 0.6249428797647325]
[(3, 5, 14), 0.6177966293405723]
[(3, 6, 7), 0.613510915054858]
[(3, 6, 8), 0.6106515100101799]
[(3, 6, 9), 0.6111299626739056]
[(3, 6, 10), 0.6139882366248163]
[(3, 6, 11), 0.6158873430607397]
[(3, 6, 12), 0.6125630584775477]
[(3, 6, 13), 0.6206515100101798]
[(3, 6, 14), 0.6139825811559778]
[(3, 7, 8), 0.6173340119895939]
[(3, 7, 9), 0.619712702183011]
[(3, 7, 10), 0.6021060965954078]
[(3, 7, 11), 0.6054315122723674]
[(3, 7, 12), 0.6159065716547902]
[(3, 7, 13), 0.6159054405610225]
[(3, 7, 14), 0.6211333559552087]
[(3, 8, 9), 0.618753534668024]
[(3, 8, 10), 0.6149462730460356]
[(3, 8, 11), 0.6097127021830111]
[(3, 8, 12), 0.6192342495192851]
[(3, 8, 13), 0.6258952607171134]
[(3, 8, 14), 0.62017984390906]
[(3, 9, 10), 0.6144723447573803]
[(3, 9, 11), 0.6144746069449158]
[(3, 9, 12), 0.6235188327

[(1, 2, 6, 14), 0.679667458432304]
[(1, 2, 7, 8), 0.6787343060739737]
[(1, 2, 7, 9), 0.6796878181201222]
[(1, 2, 7, 10), 0.6749293066395203]
[(1, 2, 7, 11), 0.6782569845040154]
[(1, 2, 7, 12), 0.6815880556498135]
[(1, 2, 7, 13), 0.6787354371677413]
[(1, 2, 7, 14), 0.6773057346454021]
[(1, 2, 8, 9), 0.6834950797421107]
[(1, 2, 8, 10), 0.6830098405157787]
[(1, 2, 8, 11), 0.6839690080307659]
[(1, 2, 8, 12), 0.6830132337970818]
[(1, 2, 8, 13), 0.6834916864608077]
[(1, 2, 8, 14), 0.6787252573238323]
[(1, 2, 9, 10), 0.6877751385589865]
[(1, 2, 9, 11), 0.6811163895486937]
[(1, 2, 9, 12), 0.6849134713267729]
[(1, 2, 9, 13), 0.682067639407307]
[(1, 2, 9, 14), 0.6777751385589866]
[(1, 2, 10, 11), 0.684439543038118]
[(1, 2, 10, 12), 0.6853828752403575]
[(1, 2, 10, 13), 0.6844372808505825]
[(1, 2, 10, 14), 0.6758669833729216]
[(1, 2, 11, 12), 0.6839588281868567]
[(1, 2, 11, 13), 0.6787320438864382]
[(1, 2, 11, 14), 0.680156090939939]
[(1, 2, 12, 13), 0.6839633525619275]
[(1, 2, 12, 14), 0.67682501

[(1, 8, 13, 14), 0.6834916864608077]
[(1, 9, 10, 11), 0.6753998416468725]
[(1, 9, 10, 12), 0.680152697658636]
[(1, 9, 10, 13), 0.6777762696527543]
[(1, 9, 10, 14), 0.6787320438864383]
[(1, 9, 11, 12), 0.6820585906571655]
[(1, 9, 11, 13), 0.6773068657391697]
[(1, 9, 11, 14), 0.6792059721750935]
[(1, 9, 12, 13), 0.6801515665648683]
[(1, 9, 12, 14), 0.6801549598461714]
[(1, 9, 13, 14), 0.6796855559325868]
[(1, 10, 11, 12), 0.6753873996154282]
[(1, 10, 11, 13), 0.672541567695962]
[(1, 10, 11, 14), 0.6711118651736229]
[(1, 10, 12, 13), 0.6772932926139578]
[(1, 10, 12, 14), 0.6744361497568148]
[(1, 10, 13, 14), 0.6725370433208914]
[(1, 11, 12, 13), 0.6863386494740414]
[(1, 11, 12, 14), 0.6782501979414093]
[(1, 11, 13, 14), 0.6801572220337067]
[(1, 12, 13, 14), 0.6811073407985523]
[(2, 3, 4, 5), 0.6349530596086416]
[(2, 3, 4, 6), 0.6349406175771971]
[(2, 3, 4, 7), 0.6292421671756588]
[(2, 3, 4, 8), 0.6373328808958262]
[(2, 3, 4, 9), 0.6382841307544396]
[(2, 3, 4, 10), 0.6416163329940052]
[(2,

[(2, 9, 11, 14), 0.5882909173170455]
[(2, 9, 12, 13), 0.5949598461712476]
[(2, 9, 12, 14), 0.5911571089243299]
[(2, 9, 13, 14), 0.5959133582173962]
[(2, 10, 11, 12), 0.5849621083587829]
[(2, 10, 11, 13), 0.5840074652188667]
[(2, 10, 11, 14), 0.5797217509331523]
[(2, 10, 12, 13), 0.5854382988349733]
[(2, 10, 12, 14), 0.5882931795045809]
[(2, 10, 13, 14), 0.5854349055536705]
[(2, 11, 12, 13), 0.5911491912679561]
[(2, 11, 12, 14), 0.5887705010745391]
[(2, 11, 13, 14), 0.5878169890283905]
[(2, 12, 13, 14), 0.5935324058364438]
[(3, 4, 5, 6), 0.636838592919353]
[(3, 4, 5, 7), 0.635432643366135]
[(3, 4, 5, 8), 0.6444678203823097]
[(3, 4, 5, 9), 0.6435165705236965]
[(3, 4, 5, 10), 0.638753534668024]
[(3, 4, 5, 11), 0.6354122836783169]
[(3, 4, 5, 12), 0.6368453794819591]
[(3, 4, 5, 13), 0.6425698450401538]
[(3, 4, 5, 14), 0.6382784752856012]
[(3, 4, 6, 7), 0.6320834747200543]
[(3, 4, 6, 8), 0.6339780567809071]
[(3, 4, 6, 9), 0.6373125212080082]
[(3, 4, 6, 10), 0.6339803189684425]
[(3, 4, 6, 11)

...

[(2, 3, 5, 6, 8, 9, 10, 11, 13, 14), 0.620166270783848]
[(2, 3, 5, 6, 8, 9, 10, 12, 13, 14), 0.6125540097274065]
[(2, 3, 5, 6, 8, 9, 11, 12, 13, 14), 0.6182626399728537]
[(2, 3, 5, 6, 8, 10, 11, 12, 13, 14), 0.614456509444633]
[(2, 3, 5, 6, 9, 10, 11, 12, 13, 14), 0.6120721637823776]
[(2, 3, 5, 7, 8, 9, 10, 11, 12, 13), 0.5987625834181654]
[(2, 3, 5, 7, 8, 9, 10, 11, 12, 14), 0.6068544282321005]
[(2, 3, 5, 7, 8, 9, 10, 11, 13, 14), 0.6092365117068206]
[(2, 3, 5, 7, 8, 9, 10, 12, 13, 14), 0.6101922859405045]
[(2, 3, 5, 7, 8, 9, 11, 12, 13, 14), 0.6111367492365117]
[(2, 3, 5, 7, 8, 10, 11, 12, 13, 14), 0.6082875240357426]
[(2, 3, 5, 7, 9, 10, 11, 12, 13, 14), 0.6040018097500284]
[(2, 3, 5, 8, 9, 10, 11, 12, 13, 14), 0.6130347245786676]
[(2, 3, 6, 7, 8, 9, 10, 11, 12, 13), 0.6082875240357426]
[(2, 3, 6, 7, 8, 9, 10, 11, 12, 14), 0.6097070467141726]
[(2, 3, 6, 7, 8, 9, 10, 11, 13, 14), 0.612081212532519]
[(2, 3, 6, 7, 8, 9, 10, 12, 13, 14), 0.6106582965727859]
[(2, 3, 6, 7, 8, 9, 11, 12, 1

[(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12), 0.7201368623458885]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13), 0.7187105531048524]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14), 0.7153840063341251]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12), 0.7253715643026807]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13), 0.7153749575839837]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 14), 0.718230969347359]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13), 0.717278588394978]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 14), 0.7163296007239001]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 13, 14), 0.7158511480601742]
[(1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12), 0.722042755344418]
[(1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13), 0.7144237077253704]
[(1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 14), 0.7115710892432983]
[(1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13), 0.7172831127700487]
[(1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 14), 0.717281981676281]
[(1, 2, 3, 4, 5, 6, 7, 8, 10, 13, 14), 0.7091946612374166]
[(1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13), 0.7206073973532406]
[(1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 14), 0.7201357312521208]
[(1, 2, 3,

[(1, 2, 3, 5, 6, 7, 10, 11, 12, 13, 14), 0.6825404366021944]
[(1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13), 0.6791991856124874]
[(1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 14), 0.6815801379934396]
[(1, 2, 3, 5, 6, 8, 9, 10, 11, 13, 14), 0.6811062097047844]
[(1, 2, 3, 5, 6, 8, 9, 10, 12, 13, 14), 0.677298948082796]
[(1, 2, 3, 5, 6, 8, 9, 11, 12, 13, 14), 0.6806277570410588]
[(1, 2, 3, 5, 6, 8, 10, 11, 12, 13, 14), 0.6811039475172492]
[(1, 2, 3, 5, 6, 9, 10, 11, 12, 13, 14), 0.6796787693699808]
[(1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13), 0.6811175206424613]
[(1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 14), 0.6787343060739737]
[(1, 2, 3, 5, 7, 8, 9, 10, 11, 13, 14), 0.6753998416468725]
[(1, 2, 3, 5, 7, 8, 9, 10, 12, 13, 14), 0.6782569845040154]
[(1, 2, 3, 5, 7, 8, 9, 11, 12, 13, 14), 0.6749270444519851]
[(1, 2, 3, 5, 7, 8, 10, 11, 12, 13, 14), 0.6763510915054859]
[(1, 2, 3, 5, 7, 9, 10, 11, 12, 13, 14), 0.679213889831467]
[(1, 2, 3, 5, 8, 9, 10, 11, 12, 13, 14), 0.678729781698903]
[(1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13

[(1, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14), 0.7153659088338424]
[(1, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14), 0.7096550164008597]
[(1, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14), 0.7120280511254384]
[(1, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14), 0.7072763262074426]
[(1, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14), 0.7082275760660559]
[(1, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14), 0.7144067413188553]
[(1, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14), 0.7091867435810428]
[(1, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14), 0.7077536477774007]
[(1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14), 0.6620642461260038]
[(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), 0.6197013912453342]
[(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13), 0.6263725822870716]
[(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14), 0.6235154394299287]
[(2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13), 0.6239871055310485]
[(2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14), 0.6258929985295781]
[(2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14), 0.62779776043434]
[(2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13), 0.6239916299061192]
[(2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 14), 0.62

[(1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 13, 14), 0.6815903178373487]
[(1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 14), 0.6825393055084268]
[(1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14), 0.6834894242732723]
[(1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14), 0.6796832937450515]
[(1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14), 0.676343173849112]
[(1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14), 0.6749247822644499]
[(1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14), 0.683487162085737]
[(1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), 0.7115643026806924]
[(1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14), 0.7106107906345436]
[(1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14), 0.7139429928741092]
[(1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14), 0.7101346001583531]
[(1, 2, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14), 0.7082230516909853]
[(1, 2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14), 0.7101346001583531]
[(1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14), 0.7110858500169664]
[(1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14), 0.7148863250763489]
[(1, 2, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14), 0.705

In [107]:
MM=max([tup[1] for tup in data_poi])
MM
#how is this smaller than .717 ?? there is a mistake
#And... how are these scores identical??

0.7334600158353128

In [108]:
best_subs=[tupe[0] for tupe in data_poi if tupe[1]==MM]
best_subs

[(1, 3, 4, 6, 7, 8)]

In [121]:
candi=[]
candi.extend(can)
for i in (1, 3, 4, 6, 7, 8):
    candi.extend(keeps[i-1])
model=LogisticRegression()
XXXX=ufc_best_df[method_mask][candi].iloc[0:2101]
yyyy=ufc_fights_clean_stance[method_mask].iloc[0:2101]['result']
cross_val_score(model,XXXX,yyyy,cv=5).mean()

0.7334600158353128

In [120]:
len(best_sub[0])+len(best_subs[0])

12

In [113]:
winners=[]
winners.extend(can)
for i in best_subs[0]:
    winners.extend(keeps[i-1])

In [122]:
winners==candi

True

In [157]:
model=LogisticRegression()
XXXX=ufc_best_df[method_mask][winners].iloc[0:2091]
yyyy=ufc_fights_clean_stance[method_mask].iloc[0:2091]['result']
cross_val_score(model,XXXX,yyyy,cv=33).mean()

0.7389457167682973

In [153]:
pts=[]
for r in range(5,50):
    model=LogisticRegression()
    XXXX=ufc_best_df[method_mask][winners].iloc[0:2091]
    yyyy=ufc_fights_clean_stance[method_mask].iloc[0:2091]['result']
    pts.append([r,cross_val_score(model,XXXX,yyyy,cv=r).mean()])

In [155]:
mm=max([pt[1] for pt in pts])

In [156]:
[pt[0] for pt in pts if pt[1]==mm]

[33]

In [158]:
#this is really not overfit at all. I think we have found quite a good set of features
XXXX=ufc_best_df[method_mask][winners].iloc[0:2091]
yyyy=ufc_fights_clean_stance[method_mask].iloc[0:2091]['result']
model.fit(XXXX,yyyy)
model.score(XXXX,yyyy)

0.7479674796747967

In [112]:
winners

['fighter_ko_wins_diff_2',
 'fighter_inf_total_strikes_landed_avg',
 'opponent_inf_total_strikes_landed_avg',
 'fighter_abs_total_strikes_landed_avg',
 'opponent_abs_total_strikes_landed_avg',
 'fighter_age',
 'opponent_age',
 'fighter_sub_wins_diff_2',
 'fighter_inf_head_strikes_landed_avg',
 'fighter_abs_head_strikes_landed_avg',
 'opponent_inf_head_strikes_landed_avg',
 'opponent_abs_head_strikes_landed_avg',
 'fighter_ko_losses_diff_2',
 'fighter_inf_ground_strikes_landed_avg',
 'fighter_abs_ground_strikes_landed_avg',
 'opponent_inf_ground_strikes_landed_avg',
 'opponent_abs_ground_strikes_landed_avg',
 'fighter_inf_body_strikes_landed_avg',
 'fighter_inf_body_strikes_attempts_avg',
 'fighter_abs_body_strikes_landed_avg',
 'fighter_abs_body_strikes_attempts_avg',
 'opponent_inf_body_strikes_landed_avg',
 'opponent_inf_body_strikes_attempts_avg',
 'opponent_abs_body_strikes_landed_avg',
 'opponent_abs_body_strikes_attempts_avg',
 'fighter_inf_takedowns_landed_avg',
 'fighter_abs_ta

In [160]:
len(maybies+keeps)

28

In [165]:
maybies_best=list(best_sub[0])

In [166]:
keeps_best=list(best_subs[0])

In [167]:
for i in range(len(keeps_best)):
    keeps_best[i]+=14

In [169]:
ultimate_master=maybies+keeps

In [170]:
best_so_far=maybies_best+keeps_best

In [176]:
best_so_far

[1, 2, 3, 4, 5, 7, 15, 17, 18, 20, 21, 22]

In [171]:
len(ultimate_master)

28

In [172]:
len(best_so_far)

12

In [177]:
ultimate_feature_list=[]
for i in best_so_far:
    ultimate_feature_list.extend(ultimate_master[i-1])

In [178]:
model=LogisticRegression()
XXXX=ufc_best_df[method_mask][ultimate_feature_list].iloc[0:2091]
yyyy=ufc_fights_clean_stance[method_mask].iloc[0:2091]['result']
cross_val_score(model,XXXX,yyyy,cv=33).mean()

0.7389457167682973

In [ ]:
#My idea was to look over all subsets of ultimate master of size 12. This would take (28 choose 12)/5/60/60 hours
#this is about 2000 hours...
#For now we seem to have hit a max.
#Later with more computational power we can possibly improve this.

In [9]:
ultimate_feature_list=['fighter_wins_diff_2',
 'fighter_losses_diff_2',
 'fighter_L5Y_wins_diff_2',
 'fighter_L5Y_losses_diff_2',
 'fighter_L2Y_wins_diff_2',
 'fighter_L2Y_losses_diff_2',
 'fighter_L5Y_ko_wins_diff_2',
 'fighter_L5Y_ko_losses_diff_2',
 'fighter_L2Y_ko_wins_diff_2',
 'fighter_L2Y_ko_losses_diff_2',
 'fighter_sub_losses_diff_2',
 'fighter_L5Y_sub_wins_diff_2',
 'fighter_L5Y_sub_losses_diff_2',
 'fighter_L2Y_sub_wins_diff_2',
 'fighter_L2Y_sub_losses_diff_2',
 'fighter_inf_reversals_avg',
 'fighter_abs_reversals_avg',
 'opponent_inf_reversals_avg',
 'opponent_abs_reversals_avg',
 'fighter_inf_takedowns_attempts_avg',
 'fighter_abs_takedowns_attempts_avg',
 'opponent_inf_takedowns_attempts_avg',
 'opponent_abs_takedowns_attempts_avg',
 'fighter_ko_wins_diff_2',
 'fighter_age',
 'opponent_age',
 'fighter_sub_wins_diff_2',
 'fighter_ko_losses_diff_2',
 'fighter_inf_body_strikes_landed_avg',
 'fighter_inf_body_strikes_attempts_avg',
 'fighter_abs_body_strikes_landed_avg',
 'fighter_abs_body_strikes_attempts_avg',
 'opponent_inf_body_strikes_landed_avg',
 'opponent_inf_body_strikes_attempts_avg',
 'opponent_abs_body_strikes_landed_avg',
 'opponent_abs_body_strikes_attempts_avg',
 'fighter_inf_takedowns_landed_avg',
 'fighter_abs_takedowns_landed_avg',
 'opponent_inf_takedowns_landed_avg',
 'opponent_abs_takedowns_landed_avg']

In [179]:
#lets remove both fighter and opponent absorbed body strikes attempts

In [10]:
modified_ultimate_feature_list=['fighter_wins_diff_2',
 'fighter_losses_diff_2',
 'fighter_L5Y_wins_diff_2',
 'fighter_L5Y_losses_diff_2',
 'fighter_L2Y_wins_diff_2',
 'fighter_L2Y_losses_diff_2',
 'fighter_L5Y_ko_wins_diff_2',
 'fighter_L5Y_ko_losses_diff_2',
 'fighter_L2Y_ko_wins_diff_2',
 'fighter_L2Y_ko_losses_diff_2',
 'fighter_sub_losses_diff_2',
 'fighter_L5Y_sub_wins_diff_2',
 'fighter_L5Y_sub_losses_diff_2',
 'fighter_L2Y_sub_wins_diff_2',
 'fighter_L2Y_sub_losses_diff_2',
 'fighter_inf_reversals_avg',
 'fighter_abs_reversals_avg',
 'opponent_inf_reversals_avg',
 'opponent_abs_reversals_avg',
 'fighter_inf_takedowns_attempts_avg',
 'fighter_abs_takedowns_attempts_avg',
 'opponent_inf_takedowns_attempts_avg',
 'opponent_abs_takedowns_attempts_avg',
 'fighter_ko_wins_diff_2',
 'fighter_age',
 'opponent_age',
 'fighter_sub_wins_diff_2',
 'fighter_ko_losses_diff_2',
 'fighter_inf_body_strikes_landed_avg',
 'fighter_inf_body_strikes_attempts_avg',
 'fighter_abs_body_strikes_landed_avg',
 'opponent_inf_body_strikes_landed_avg',
 'opponent_inf_body_strikes_attempts_avg',
 'opponent_abs_body_strikes_landed_avg',
 'fighter_inf_takedowns_landed_avg',
 'fighter_abs_takedowns_landed_avg',
 'opponent_inf_takedowns_landed_avg',
 'opponent_abs_takedowns_landed_avg']

In [187]:
#This gets us very close to 74% 
model=LogisticRegression()
XXXX=ufc_best_df[method_mask][modified_ultimate_feature_list].iloc[0:2091]
yyyy=ufc_fights_clean_stance[method_mask].iloc[0:2091]['result']
cross_val_score(model,XXXX,yyyy,cv=33).mean()

0.7398701395677202

In [22]:
keepers=['fighter_ko_wins_diff_2',
        'fighter_inf_total_strikes_landed_avg',
        'opponent_inf_total_strikes_landed_avg',
        'fighter_abs_total_strikes_landed_avg',
        'opponent_abs_total_strikes_landed_avg',
        'fighter_age',
        'opponent_age',
        'fighter_sub_wins_diff_2',
        'fighter_inf_head_strikes_landed_avg',
        'fighter_abs_head_strikes_landed_avg',
        'opponent_inf_head_strikes_landed_avg',
        'opponent_abs_head_strikes_landed_avg',
        'fighter_ko_losses_diff_2',
        'fighter_inf_ground_strikes_landed_avg',
        'fighter_abs_ground_strikes_landed_avg',
        'opponent_inf_ground_strikes_landed_avg',
        'opponent_abs_ground_strikes_landed_avg',
        'fighter_inf_body_strikes_landed_avg',
        'fighter_inf_body_strikes_attempts_avg',
        'fighter_abs_body_strikes_landed_avg',
        'fighter_abs_body_strikes_attempts_avg',
        'opponent_inf_body_strikes_landed_avg',
        'opponent_inf_body_strikes_attempts_avg',
        'opponent_abs_body_strikes_landed_avg',
        'opponent_abs_body_strikes_attempts_avg',
        'fighter_inf_takedowns_landed_avg',
        'fighter_abs_takedowns_landed_avg',
        'opponent_inf_takedowns_landed_avg',
        'opponent_abs_takedowns_landed_avg',
        'fighter_inf_knockdowns_avg',
        'fighter_abs_knockdowns_avg',
        'opponent_inf_knockdowns_avg',
        'opponent_abs_knockdowns_avg',
        'fighter_inf_ground_strikes_attempts_avg',
        'fighter_abs_ground_strikes_attempts_avg',
        'opponent_inf_ground_strikes_attempts_avg',
        'opponent_abs_ground_strikes_attempts_avg',
        'fighter_inf_leg_strikes_landed_avg',
        'fighter_inf_leg_strikes_attempts_avg',
        'fighter_abs_leg_strikes_landed_avg',
        'fighter_abs_leg_strikes_attempts_avg',
        'opponent_inf_leg_strikes_landed_avg',
        'opponent_inf_leg_strikes_attempts_avg',
        'opponent_abs_leg_strikes_landed_avg',
        'opponent_abs_leg_strikes_attempts_avg',
        'fighter_inf_clinch_strikes_attempts_avg',
        'fighter_abs_clinch_strikes_attempts_avg',
        'opponent_inf_clinch_strikes_attempts_avg',
        'opponent_abs_clinch_strikes_attempts_avg',
        'fighter_inf_pass_avg',
        'fighter_abs_pass_avg',
        'opponent_inf_pass_avg',
        'opponent_abs_pass_avg',
        'fighter_stance',
        'opponent_stance',
        'height_diff',
        'reach_diff']

mlp_maybies=[
['fighter_wins_diff_2',
'fighter_losses_diff_2',
 'fighter_L5Y_wins_diff_2',
'fighter_L5Y_losses_diff_2',
'fighter_L2Y_wins_diff_2',
'fighter_L2Y_losses_diff_2'],
['fighter_L5Y_ko_wins_diff_2',
'fighter_L5Y_ko_losses_diff_2',
'fighter_L2Y_ko_wins_diff_2',
'fighter_L2Y_ko_losses_diff_2'],
['fighter_sub_losses_diff_2',
'fighter_L5Y_sub_wins_diff_2',
'fighter_L5Y_sub_losses_diff_2',
'fighter_L2Y_sub_wins_diff_2',
'fighter_L2Y_sub_losses_diff_2'],
['fighter_inf_reversals_avg',
'fighter_abs_reversals_avg',
'opponent_inf_reversals_avg',
'opponent_abs_reversals_avg'],
['fighter_inf_sub_attempts_avg',
'fighter_abs_sub_attempts_avg',
'opponent_inf_sub_attempts_avg',
'opponent_abs_sub_attempts_avg'],
['fighter_inf_takedowns_attempts_avg',
'fighter_abs_takedowns_attempts_avg',
'opponent_inf_takedowns_attempts_avg',
'opponent_abs_takedowns_attempts_avg'],
['fighter_inf_sig_strikes_landed_avg',
'fighter_abs_sig_strikes_landed_avg',
'opponent_inf_sig_strikes_landed_avg',
'opponent_abs_sig_strikes_landed_avg',
 'fighter_inf_sig_strikes_attempts_avg',
'fighter_abs_sig_strikes_attempts_avg',
'opponent_inf_sig_strikes_attempts_avg',
'opponent_abs_sig_strikes_attempts_avg'],
['fighter_inf_total_strikes_attempts_avg',
'fighter_abs_total_strikes_attempts_avg',
'opponent_inf_total_strikes_attempts_avg',
'opponent_abs_total_strikes_attempts_avg'],
['fighter_inf_head_strikes_attempts_avg',
'fighter_abs_head_strikes_attempts_avg',
'opponent_inf_head_strikes_attempts_avg',
'opponent_abs_head_strikes_attempts_avg'],
['fighter_inf_distance_strikes_landed_avg',
'fighter_abs_distance_strikes_landed_avg',
'opponent_inf_distance_strikes_landed_avg',
'opponent_abs_distance_strikes_landed_avg',
 'fighter_inf_distance_strikes_attempts_avg',
'fighter_abs_distance_strikes_attempts_avg',
'opponent_inf_distance_strikes_attempts_avg',
'opponent_abs_distance_strikes_attempts_avg'],
['fighter_inf_clinch_strikes_landed_avg',
'fighter_abs_clinch_strikes_landed_avg',
'opponent_inf_clinch_strikes_landed_avg',
'opponent_abs_clinch_strikes_landed_avg'],
]
len(mlp_maybies)

11

In [22]:
mlp=MLPClassifier(max_iter=1000,alpha=.001,solver='sgd',random_state=3)
ypp=ufc_fights_clean_stance[method_mask].iloc[0:2101]['method']
mlp_data_points=[]
SUB=subsets(11)
for S in SUB:
    candidate=[]
    candidate.extend(keepers)
    for i in S:
        candidate.extend(mlp_maybies[i-1])
    Xpp=ufc_best_df[method_mask][candidate].iloc[0:2101]
    point=[S,cross_val_score(mlp,Xpp,ypp,cv=3).mean()]
    mlp_data_points.append(point)
    print(point)

[[], 0.4797642823177773]
[(1,), 0.4683329936824944]
[(2,), 0.4583309557774608]
[(3,), 0.4459581550166429]
[(4,), 0.5069010257455336]
[(5,), 0.5007098702533795]
[(6,), 0.5026132735547856]
[(7,), 0.4930935398410434]
[(8,), 0.49547313361864]
[(9,), 0.49404999660349164]
[(10,), 0.44309489844439914]
[(11,), 0.5059445689830854]
[(1, 2), 0.4992846953331975]
[(1, 3), 0.503564295903811]
[(1, 4), 0.48880442904693977]
[(1, 5), 0.47689966714217785]
[(1, 6), 0.49071054955505744]
[(1, 7), 0.4902377555872563]
[(1, 8), 0.493090143332654]
[(1, 9), 0.4878506894912031]
[(1, 10), 0.48691257387405745]
[(1, 11), 0.49689830853882205]
[(2, 3), 0.5078411792677128]
[(2, 4), 0.4973846885401807]
[(2, 5), 0.49690849806399023]
[(2, 6), 0.4359615515250323]
[(2, 7), 0.49071938047686975]
[(2, 8), 0.4921520277155085]
[(2, 9), 0.43643910060457847]
[(2, 10), 0.49785136879288094]
[(2, 11), 0.3988112220637185]
[(3, 4), 0.4921513484138306]
[(3, 5), 0.4959581550166429]
[(3, 6), 0.4940540724135589]
[(3, 7), 0.4764241559676652

[(1, 2, 6, 9), 0.416430948984444]
[(1, 2, 6, 10), 0.48786019971469335]
[(1, 2, 6, 11), 0.43832755926907135]
[(1, 2, 7, 8), 0.48786155831804906]
[(1, 2, 7, 9), 0.48642075945927593]
[(1, 2, 7, 10), 0.49452822498471577]
[(1, 2, 7, 11), 0.47881597717546365]
[(1, 2, 8, 9), 0.49452482847632634]
[(1, 2, 8, 10), 0.4864357040961891]
[(1, 2, 8, 11), 0.47975884790435436]
[(1, 2, 9, 10), 0.4873819713334692]
[(1, 2, 9, 11), 0.49118741933292576]
[(1, 2, 10, 11), 0.44358263704911355]
[(1, 3, 4, 5), 0.49690578085727877]
[(1, 3, 4, 6), 0.48880918415868485]
[(1, 3, 4, 7), 0.4954799266354188]
[(1, 3, 4, 8), 0.5026024047279397]
[(1, 3, 4, 9), 0.4954724543169622]
[(1, 3, 4, 10), 0.5030989742544665]
[(1, 3, 4, 11), 0.4807159839684804]
[(1, 3, 5, 6), 0.49356837171387813]
[(1, 3, 5, 7), 0.49691121527070176]
[(1, 3, 5, 8), 0.511170436790979]
[(1, 3, 5, 9), 0.4954744922219958]
[(1, 3, 5, 10), 0.49786223761972687]
[(1, 3, 5, 11), 0.48643706269954484]
[(1, 3, 6, 7), 0.49643162828612186]
[(1, 3, 6, 8), 0.481670402

[(3, 7, 8, 10), 0.4807207390802255]
[(3, 7, 8, 11), 0.48453773520820603]
[(3, 7, 9, 10), 0.48786223761972697]
[(3, 7, 9, 11), 0.4288254873989539]
[(3, 7, 10, 11), 0.49785680320630393]
[(3, 8, 9, 10), 0.47977175463623395]
[(3, 8, 9, 11), 0.46928401603151954]
[(3, 8, 10, 11), 0.4926119149514299]
[(3, 9, 10, 11), 0.5059432103797296]
[(4, 5, 6, 7), 0.4935738061273012]
[(4, 5, 6, 8), 0.49071258746009105]
[(4, 5, 6, 9), 0.48643162828612185]
[(4, 5, 6, 10), 0.4892867332382311]
[(4, 5, 6, 11), 0.4811935330480266]
[(4, 5, 7, 8), 0.48833978669927314]
[(4, 5, 7, 9), 0.49453094219142724]
[(4, 5, 7, 10), 0.4845085252360574]
[(4, 5, 7, 11), 0.49833978669927315]
[(4, 5, 8, 9), 0.5011928537463488]
[(4, 5, 8, 10), 0.49024998301745804]
[(4, 5, 8, 11), 0.5068983085388221]
[(4, 5, 9, 10), 0.49738332993682494]
[(4, 5, 9, 11), 0.4873833299368249]
[(4, 5, 10, 11), 0.48119421234970455]
[(4, 6, 7, 8), 0.49357924054072416]
[(4, 6, 7, 9), 0.48214795190544124]
[(4, 6, 7, 10), 0.4692853746348753]
[(4, 6, 7, 11), 0

[(1, 3, 8, 9, 11), 0.49024998301745804]
[(1, 3, 8, 10, 11), 0.49452550777800425]
[(1, 3, 9, 10, 11), 0.4645289042863936]
[(1, 4, 5, 6, 7), 0.4769159703824469]
[(1, 4, 5, 6, 8), 0.49499898104748313]
[(1, 4, 5, 6, 9), 0.4897534134909313]
[(1, 4, 5, 6, 10), 0.46597242035187825]
[(1, 4, 5, 6, 11), 0.4873833299368249]
[(1, 4, 5, 7, 8), 0.4902533795258474]
[(1, 4, 5, 7, 9), 0.48977379254126757]
[(1, 4, 5, 7, 10), 0.4930935398410434]
[(1, 4, 5, 7, 11), 0.4916765165409958]
[(1, 4, 5, 8, 9), 0.48022960396712183]
[(1, 4, 5, 8, 10), 0.4945289042863936]
[(1, 4, 5, 8, 11), 0.48117926771279124]
[(1, 4, 5, 9, 10), 0.5002472658107465]
[(1, 4, 5, 9, 11), 0.48833027647578287]
[(1, 4, 5, 10, 11), 0.5026112356497521]
[(1, 4, 6, 7, 8), 0.4916792337477074]
[(1, 4, 6, 7, 9), 0.4850186807961416]
[(1, 4, 6, 7, 10), 0.4959459275864411]
[(1, 4, 6, 7, 11), 0.5002323211738332]
[(1, 4, 6, 8, 9), 0.5026085184430406]
[(1, 4, 6, 8, 10), 0.4854853610488418]
[(1, 4, 6, 8, 11), 0.4978452550777801]
[(1, 4, 6, 9, 10), 0.47

[(3, 4, 5, 8, 11), 0.4935758440323348]
[(3, 4, 5, 9, 10), 0.4911996467631275]
[(3, 4, 5, 9, 11), 0.49215542422389785]
[(3, 4, 5, 10, 11), 0.4797629237144216]
[(3, 4, 6, 7, 8), 0.4935731268256232]
[(3, 4, 6, 7, 9), 0.4840506759051695]
[(3, 4, 6, 7, 10), 0.48643977990625636]
[(3, 4, 6, 7, 11), 0.49689762923714426]
[(3, 4, 6, 8, 9), 0.4973812920317913]
[(3, 4, 6, 8, 10), 0.48453298009646084]
[(3, 4, 6, 8, 11), 0.48595883431832076]
[(3, 4, 6, 9, 10), 0.4911996467631275]
[(3, 4, 6, 9, 11), 0.4869207254941919]
[(3, 4, 6, 10, 11), 0.4816717614292507]
[(3, 4, 7, 8, 9), 0.47787174784321723]
[(3, 4, 7, 8, 10), 0.4792948848583656]
[(3, 4, 7, 8, 11), 0.5049860743156036]
[(3, 4, 7, 9, 10), 0.4940574689219483]
[(3, 4, 7, 9, 11), 0.49070375653827863]
[(3, 4, 7, 10, 11), 0.47405611031859246]
[(3, 4, 8, 9, 10), 0.5016656477141498]
[(3, 4, 8, 9, 11), 0.4864411385096121]
[(3, 4, 8, 10, 11), 0.4992812988248081]
[(3, 4, 9, 10, 11), 0.4940391277766456]
[(3, 5, 6, 7, 8), 0.49690578085727877]
[(3, 5, 6, 7, 9)

...

[(2, 3, 4, 7, 10, 11), 0.4812003260648054]
[(2, 3, 4, 8, 9, 10), 0.4926214251749202]
[(2, 3, 4, 8, 9, 11), 0.49737925412675765]
[(2, 3, 4, 8, 10, 11), 0.4964370626995449]
[(2, 3, 4, 9, 10, 11), 0.4897669995244888]
[(2, 3, 5, 6, 7, 8), 0.4854846817471639]
[(2, 3, 5, 6, 7, 9), 0.4797894164798587]
[(2, 3, 5, 6, 7, 10), 0.48168263025609676]
[(2, 3, 5, 6, 7, 11), 0.46405339311188093]
[(2, 3, 5, 6, 8, 9), 0.49880714625365125]
[(2, 3, 5, 6, 8, 10), 0.4911982881597718]
[(2, 3, 5, 6, 8, 11), 0.4907119081584132]
[(2, 3, 5, 6, 9, 10), 0.49073024930371584]
[(2, 3, 5, 6, 9, 11), 0.5083173697439033]
[(2, 3, 5, 6, 10, 11), 0.5016608926024047]
[(2, 3, 5, 7, 8, 9), 0.4864302696827661]
[(2, 3, 5, 7, 8, 10), 0.475022077304531]
[(2, 3, 5, 7, 8, 11), 0.49596087222335433]
[(2, 3, 5, 7, 9, 10), 0.48882820460566534]
[(2, 3, 5, 7, 9, 11), 0.4888180150804972]
[(2, 3, 5, 7, 10, 11), 0.48929556416004344]
[(2, 3, 5, 8, 9, 10), 0.4907227769852591]
[(2, 3, 5, 8, 9, 11), 0.5007091909517016]
[(2, 3, 5, 8, 10, 11), 0.4

[(1, 2, 3, 4, 5, 9, 10), 0.5049921880307044]
[(1, 2, 3, 4, 5, 9, 11), 0.47357720263569053]
[(1, 2, 3, 4, 5, 10, 11), 0.5035629373004551]
[(1, 2, 3, 4, 6, 7, 8), 0.4697622444127438]
[(1, 2, 3, 4, 6, 7, 9), 0.47025134162081383]
[(1, 2, 3, 4, 6, 7, 10), 0.5049853950139257]
[(1, 2, 3, 4, 6, 7, 11), 0.48929828136675496]
[(1, 2, 3, 4, 6, 8, 9), 0.48308199171251953]
[(1, 2, 3, 4, 6, 8, 10), 0.45262482168330953]
[(1, 2, 3, 4, 6, 8, 11), 0.48785068949120297]
[(1, 2, 3, 4, 6, 9, 10), 0.5045187147612255]
[(1, 2, 3, 4, 6, 9, 11), 0.4678601997146932]
[(1, 2, 3, 4, 6, 10, 11), 0.4602397934922899]
[(1, 2, 3, 4, 7, 8, 9), 0.48690442225392294]
[(1, 2, 3, 4, 7, 8, 10), 0.4683425039059846]
[(1, 2, 3, 4, 7, 8, 11), 0.49929080904829837]
[(1, 2, 3, 4, 7, 9, 10), 0.44358603355750287]
[(1, 2, 3, 4, 7, 9, 11), 0.47406018612865974]
[(1, 2, 3, 4, 7, 10, 11), 0.4878275932341553]
[(1, 2, 3, 4, 8, 9, 10), 0.49215270701718633]
[(1, 2, 3, 4, 8, 9, 11), 0.4869003464438557]
[(1, 2, 3, 4, 8, 10, 11), 0.4916731200326065]

[(1, 4, 5, 7, 8, 9, 10), 0.49642279736430944]
[(1, 4, 5, 7, 8, 9, 11), 0.49643162828612186]
[(1, 4, 5, 7, 8, 10, 11), 0.4864289110794104]
[(1, 4, 5, 7, 9, 10, 11), 0.4954697371102506]
[(1, 4, 5, 8, 9, 10, 11), 0.4792914883499762]
[(1, 4, 6, 7, 8, 9, 10), 0.4807119081584132]
[(1, 4, 6, 7, 8, 9, 11), 0.48834725901772974]
[(1, 4, 6, 7, 8, 10, 11), 0.49357856123904625]
[(1, 4, 6, 7, 9, 10, 11), 0.48642619387269886]
[(1, 4, 6, 8, 9, 10, 11), 0.49785884111133755]
[(1, 4, 7, 8, 9, 10, 11), 0.4631166360980912]
[(1, 5, 6, 7, 8, 9, 10), 0.4973765369200462]
[(1, 5, 6, 7, 8, 9, 11), 0.47834657971605193]
[(1, 5, 6, 7, 8, 10, 11), 0.4954643026968277]
[(1, 5, 6, 7, 9, 10, 11), 0.488803070443584]
[(1, 5, 6, 8, 9, 10, 11), 0.4940574689219483]
[(1, 5, 7, 8, 9, 10, 11), 0.47786291692140476]
[(1, 6, 7, 8, 9, 10, 11), 0.4821425174920182]
[(2, 3, 4, 5, 6, 7, 8), 0.47977243393791186]
[(2, 3, 4, 5, 6, 7, 9), 0.46500917057265134]
[(2, 3, 4, 5, 6, 7, 10), 0.49214998981047486]
[(2, 3, 4, 5, 6, 7, 11), 0.46833842

[(1, 2, 3, 5, 6, 7, 9, 11), 0.4716744786359623]
[(1, 2, 3, 5, 6, 7, 10, 11), 0.46359690238434886]
[(1, 2, 3, 5, 6, 8, 9, 10), 0.4826085184430406]
[(1, 2, 3, 5, 6, 8, 9, 11), 0.4826261802866652]
[(1, 2, 3, 5, 6, 8, 10, 11), 0.4935642959038109]
[(1, 2, 3, 5, 6, 9, 10, 11), 0.4816588546973711]
[(1, 2, 3, 5, 7, 8, 9, 10), 0.4892880918415868]
[(1, 2, 3, 5, 7, 8, 9, 11), 0.48974322396576314]
[(1, 2, 3, 5, 7, 8, 10, 11), 0.4683662794647103]
[(1, 2, 3, 5, 7, 9, 10, 11), 0.47929963997011077]
[(1, 2, 3, 5, 8, 9, 10, 11), 0.5045044494259902]
[(1, 2, 3, 6, 7, 8, 9, 10), 0.48738536784185854]
[(1, 2, 3, 6, 7, 8, 9, 11), 0.5007023979349229]
[(1, 2, 3, 6, 7, 8, 10, 11), 0.47931254670199036]
[(1, 2, 3, 6, 7, 9, 10, 11), 0.4702547381292032]
[(1, 2, 3, 6, 8, 9, 10, 11), 0.4888152978737857]
[(1, 2, 3, 7, 8, 9, 10, 11), 0.4469125738740574]
[(1, 2, 4, 5, 6, 7, 8, 9), 0.4778710685415393]
[(1, 2, 4, 5, 6, 7, 8, 10), 0.4859595136199986]
[(1, 2, 4, 5, 6, 7, 8, 11), 0.4740479586984579]
[(1, 2, 4, 5, 6, 7, 9, 10)

[(1, 3, 4, 5, 6, 8, 9, 10, 11), 0.46406018612865974]
[(1, 3, 4, 5, 7, 8, 9, 10, 11), 0.4959669859384553]
[(1, 3, 4, 6, 7, 8, 9, 10, 11), 0.498340466000951]
[(1, 3, 5, 6, 7, 8, 9, 10, 11), 0.4978663134297942]
[(1, 4, 5, 6, 7, 8, 9, 10, 11), 0.49784457577610214]
[(2, 3, 4, 5, 6, 7, 8, 9, 10), 0.49881393927043]
[(2, 3, 4, 5, 6, 7, 8, 9, 11), 0.4726241423816317]
[(2, 3, 4, 5, 6, 7, 8, 10, 11), 0.48833639019088376]
[(2, 3, 4, 5, 6, 7, 9, 10, 11), 0.4897629237144216]
[(2, 3, 4, 5, 6, 8, 9, 10, 11), 0.485962910128388]
[(2, 3, 4, 5, 7, 8, 9, 10, 11), 0.44263365260512194]
[(2, 3, 4, 6, 7, 8, 9, 10, 11), 0.435968344541811]
[(2, 3, 5, 6, 7, 8, 9, 10, 11), 0.43691800828748045]
[(2, 4, 5, 6, 7, 8, 9, 10, 11), 0.4969003464438557]
[(3, 4, 5, 6, 7, 8, 9, 10, 11), 0.461664968412472]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), 0.47784525507778003]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 11), 0.49833163507913864]
[(1, 2, 3, 4, 5, 6, 7, 8, 10, 11), 0.43502683241627604]
[(1, 2, 3, 4, 5, 6, 7, 9, 10, 11), 0.45597649616194547]
[

In [27]:
M=max([tup[1] for tup in mlp_data_points])
mlp_best_sub=[tupe[0] for tupe in mlp_data_points if tupe[1]==M]

In [24]:
M

0.5187955981251274

In [28]:
mlp_best_sub

[(2, 3, 4, 5, 6, 8, 11)]

In [29]:
mlp=MLPClassifier(max_iter=1000,alpha=.001,solver='sgd',random_state=3)
ypp=ufc_fights_clean_stance[method_mask].iloc[0:2101]['method']
candidate=[]
candidate.extend(keepers)
for i in (2, 3, 4, 5, 6, 8, 11):
    candidate.extend(mlp_maybies[i-1])
Xpp=ufc_best_df[method_mask][candidate].iloc[0:2101]
cross_val_score(mlp,Xpp,ypp,cv=3).mean()

0.5187955981251274

In [23]:
mlp_can=[]
for i in (2, 3, 4, 5, 6, 8, 11):
    mlp_can.extend(mlp_maybies[i-1])

In [24]:
mlp_keeps=[
        ['fighter_inf_total_strikes_landed_avg',
        'opponent_inf_total_strikes_landed_avg',
        'fighter_abs_total_strikes_landed_avg',
        'opponent_abs_total_strikes_landed_avg'],
        ['fighter_inf_head_strikes_landed_avg',
        'fighter_abs_head_strikes_landed_avg',
        'opponent_inf_head_strikes_landed_avg',
        'opponent_abs_head_strikes_landed_avg'],
        ['fighter_ko_wins_diff_2','fighter_ko_losses_diff_2','fighter_sub_wins_diff_2'],
        ['fighter_inf_body_strikes_landed_avg',
        'fighter_inf_body_strikes_attempts_avg',
        'fighter_abs_body_strikes_landed_avg',
        'fighter_abs_body_strikes_attempts_avg',
        'opponent_inf_body_strikes_landed_avg',
        'opponent_inf_body_strikes_attempts_avg',
        'opponent_abs_body_strikes_landed_avg',
        'opponent_abs_body_strikes_attempts_avg'],
        ['fighter_inf_takedowns_landed_avg',
        'fighter_abs_takedowns_landed_avg',
        'opponent_inf_takedowns_landed_avg',
        'opponent_abs_takedowns_landed_avg'],
        ['fighter_inf_knockdowns_avg',
        'fighter_abs_knockdowns_avg',
        'opponent_inf_knockdowns_avg',
        'opponent_abs_knockdowns_avg'],
        ['fighter_inf_ground_strikes_attempts_avg',
        'fighter_abs_ground_strikes_attempts_avg',
        'opponent_inf_ground_strikes_attempts_avg',
        'opponent_abs_ground_strikes_attempts_avg',
        'fighter_inf_ground_strikes_landed_avg',
        'fighter_abs_ground_strikes_landed_avg',
        'opponent_inf_ground_strikes_landed_avg',
        'opponent_abs_ground_strikes_landed_avg'],
        ['fighter_inf_leg_strikes_landed_avg',
        'fighter_inf_leg_strikes_attempts_avg',
        'fighter_abs_leg_strikes_landed_avg',
        'fighter_abs_leg_strikes_attempts_avg',
        'opponent_inf_leg_strikes_landed_avg',
        'opponent_inf_leg_strikes_attempts_avg',
        'opponent_abs_leg_strikes_landed_avg',
        'opponent_abs_leg_strikes_attempts_avg'],
        ['fighter_inf_clinch_strikes_attempts_avg',
        'fighter_abs_clinch_strikes_attempts_avg',
        'opponent_inf_clinch_strikes_attempts_avg',
        'opponent_abs_clinch_strikes_attempts_avg'],
        ['fighter_inf_pass_avg',
        'fighter_abs_pass_avg',
        'opponent_inf_pass_avg',
        'opponent_abs_pass_avg'],
        ['fighter_stance',
        'opponent_stance',
        'height_diff',
        'reach_diff','fighter_age',
        'opponent_age']
          ]

len(mlp_keeps)

11

In [14]:
mlp=MLPClassifier(max_iter=2000,alpha=.001,solver='sgd',random_state=3)
yq=ufc_fights_clean_stance[method_mask].iloc[0:2101]['method']
mlp_data_poi=[]
SUB=subsets(11)
for S in SUB:
    candi=[]
    candi.extend(mlp_can)
    for i in S:
        candi.extend(mlp_keeps[i-1])
    Xq=ufc_best_df[method_mask][candi].iloc[0:2101]
    point=[S,cross_val_score(mlp,Xq,yq,cv=3).mean()]
    mlp_data_poi.append(point)
    print(point)

[[], 0.5011921744446708]
[(1,), 0.4959581550166428]
[(2,), 0.5045241491746485]
[(3,), 0.4906860946946539]
[(4,), 0.49596019292167653]
[(5,), 0.4654710957136064]
[(6,), 0.4711853814278922]
[(7,), 0.49405407241355886]
[(8,), 0.49691257387405746]
[(9,), 0.49548196454045246]
[(10,), 0.4697622444127438]
[(11,), 0.4954778887303852]
[(1, 2), 0.5049962638407717]
[(1, 3), 0.5130724814890293]
[(1, 4), 0.45785952041301536]
[(1, 5), 0.4969071394606344]
[(1, 6), 0.5007146253651246]
[(1, 7), 0.4769139324774132]
[(1, 8), 0.4621425174920182]
[(1, 9), 0.49167583723931796]
[(1, 10), 0.49690849806399023]
[(1, 11), 0.4750132463827186]
[(2, 3), 0.4992554853610489]
[(2, 4), 0.49928469533319747]
[(2, 5), 0.4997642823177773]
[(2, 6), 0.49310237076285574]
[(2, 7), 0.49880714625365125]
[(2, 8), 0.503090143332654]
[(2, 9), 0.4945302628897494]
[(2, 10), 0.4959581550166429]
[(2, 11), 0.48881461857210784]
[(3, 4), 0.4654446029481692]
[(3, 5), 0.519260240472794]
[(3, 6), 0.5187854085999593]
[(3, 7), 0.44687589158345

[(1, 2, 6, 10), 0.4869159703824468]
[(1, 2, 6, 11), 0.48452143196793696]
[(1, 2, 7, 8), 0.48929556416004344]
[(1, 2, 7, 9), 0.42880646695197333]
[(1, 2, 7, 10), 0.4683357108892059]
[(1, 2, 7, 11), 0.49406018612865976]
[(1, 2, 8, 9), 0.46166836492086133]
[(1, 2, 8, 10), 0.4650037361592283]
[(1, 2, 8, 11), 0.4607220976835813]
[(1, 2, 9, 10), 0.47213776238027316]
[(1, 2, 9, 11), 0.48499966034916103]
[(1, 2, 10, 11), 0.4645241491746485]
[(1, 3, 4, 5), 0.43686909856667344]
[(1, 3, 4, 6), 0.4354473201548808]
[(1, 3, 4, 7), 0.5044956185041777]
[(1, 3, 4, 8), 0.49734664764621966]
[(1, 3, 4, 9), 0.4373459683445418]
[(1, 3, 4, 10), 0.38687045717002916]
[(1, 3, 4, 11), 0.4773663473948781]
[(1, 3, 5, 6), 0.44163236193193395]
[(1, 3, 5, 7), 0.37687589158345225]
[(1, 3, 5, 8), 0.43306432986889476]
[(1, 3, 5, 9), 0.4654425650431357]
[(1, 3, 5, 10), 0.4706752258678079]
[(1, 3, 5, 11), 0.45405678962027035]
[(1, 3, 6, 7), 0.37116364377420014]
[(1, 3, 6, 8), 0.4363935873921609]
[(1, 3, 6, 9), 0.464971129

[(3, 7, 8, 11), 0.4754812852387745]
[(3, 7, 9, 10), 0.5316289654235445]
[(3, 7, 9, 11), 0.5002187351402757]
[(3, 7, 10, 11), 0.5006983221248557]
[(3, 8, 9, 10), 0.5240146729162422]
[(3, 8, 9, 11), 0.5097377895523402]
[(3, 8, 10, 11), 0.5002146593302086]
[(3, 9, 10, 11), 0.48787514435160656]
[(4, 5, 6, 7), 0.46595883431832075]
[(4, 5, 6, 8), 0.48072141838190335]
[(4, 5, 6, 9), 0.41118606072957004]
[(4, 5, 6, 10), 0.41737993342843555]
[(4, 5, 6, 11), 0.4535683717138781]
[(4, 5, 7, 8), 0.47786019971469323]
[(4, 5, 7, 9), 0.4150118877793629]
[(4, 5, 7, 10), 0.39976903742952247]
[(4, 5, 7, 11), 0.4469091773656681]
[(4, 5, 8, 9), 0.47214455539705186]
[(4, 5, 8, 10), 0.4859581550166429]
[(4, 5, 8, 11), 0.44978262346307996]
[(4, 5, 9, 10), 0.4145187147612255]
[(4, 5, 9, 11), 0.4983370694925617]
[(4, 5, 10, 11), 0.4264282317777324]
[(4, 6, 7, 8), 0.45833503158752803]
[(4, 6, 7, 9), 0.40310848447795666]
[(4, 6, 7, 10), 0.3954833231438082]
[(4, 6, 7, 11), 0.43787038923986144]
[(4, 6, 8, 9), 0.496

[(1, 3, 8, 10, 11), 0.48406222403369337]
[(1, 3, 9, 10, 11), 0.5016513823789145]
[(1, 4, 5, 6, 7), 0.4740608654303376]
[(1, 4, 5, 6, 8), 0.4531010121595]
[(1, 4, 5, 6, 9), 0.4373867264452143]
[(1, 4, 5, 6, 10), 0.44691257387405753]
[(1, 4, 5, 6, 11), 0.372622783778276]
[(1, 4, 5, 7, 8), 0.37786087901637117]
[(1, 4, 5, 7, 9), 0.4535867128591808]
[(1, 4, 5, 7, 10), 0.45501324638271856]
[(1, 4, 5, 7, 11), 0.4688220908905645]
[(1, 4, 5, 8, 9), 0.44454520752666254]
[(1, 4, 5, 8, 10), 0.45596766524013316]
[(1, 4, 5, 8, 11), 0.47309014333265403]
[(1, 4, 5, 9, 10), 0.4340547517152367]
[(1, 4, 5, 9, 11), 0.43691868758915836]
[(1, 4, 5, 10, 11), 0.4373982745737382]
[(1, 4, 6, 7, 8), 0.38357856123904627]
[(1, 4, 6, 7, 9), 0.44930303647850006]
[(1, 4, 6, 7, 10), 0.45024862441410224]
[(1, 4, 6, 7, 11), 0.4854982677807214]
[(1, 4, 6, 8, 9), 0.4535921472726037]
[(1, 4, 6, 8, 10), 0.4526295767950547]
[(1, 4, 6, 8, 11), 0.4721377623802731]
[(1, 4, 6, 9, 10), 0.4388119013653964]
[(1, 4, 6, 9, 11), 0.442

[(3, 4, 5, 8, 11), 0.5064241559676653]
[(3, 4, 5, 9, 10), 0.4792554853610489]
[(3, 4, 5, 9, 11), 0.4978595204130154]
[(3, 4, 5, 10, 11), 0.5097479790775084]
[(3, 4, 6, 7, 8), 0.5278174037089871]
[(3, 4, 6, 7, 9), 0.5173547992663542]
[(3, 4, 6, 7, 10), 0.4797214863120712]
[(3, 4, 6, 7, 11), 0.5059391345696623]
[(3, 4, 6, 8, 9), 0.4987772569798247]
[(3, 4, 6, 8, 10), 0.49830242510699]
[(3, 4, 6, 8, 11), 0.45545683037837104]
[(3, 4, 6, 9, 10), 0.47592011412268187]
[(3, 4, 6, 9, 11), 0.5002363969839005]
[(3, 4, 6, 10, 11), 0.48879559812512735]
[(3, 4, 7, 8, 9), 0.5202092249167856]
[(3, 4, 7, 8, 10), 0.5178316690442225]
[(3, 4, 7, 8, 11), 0.48119489165138235]
[(3, 4, 7, 9, 10), 0.476866381359962]
[(3, 4, 7, 9, 11), 0.47452822498471575]
[(3, 4, 7, 10, 11), 0.5059398138713402]
[(3, 4, 8, 9, 10), 0.5325861014876707]
[(3, 4, 8, 9, 11), 0.5016615719040826]
[(3, 4, 8, 10, 11), 0.506414645744175]
[(3, 4, 9, 10, 11), 0.49880714625365125]
[(3, 5, 6, 7, 8), 0.5230622919638611]
[(3, 5, 6, 7, 9), 0.454

...

[(2, 3, 4, 7, 10, 11), 0.4821561035255757]
[(2, 3, 4, 8, 9, 10), 0.5259160383126146]
[(2, 3, 4, 8, 9, 11), 0.4764357040961891]
[(2, 3, 4, 8, 10, 11), 0.4849969431424495]
[(2, 3, 4, 9, 10, 11), 0.4649962638407717]
[(2, 3, 5, 6, 7, 8), 0.5035425582501188]
[(2, 3, 5, 6, 7, 9), 0.5325888186943822]
[(2, 3, 5, 6, 7, 10), 0.518312614632158]
[(2, 3, 5, 6, 7, 11), 0.49451939406290335]
[(2, 3, 5, 6, 8, 9), 0.4511677195842674]
[(2, 3, 5, 6, 8, 10), 0.5292588818694383]
[(2, 3, 5, 6, 8, 11), 0.4969051015556008]
[(2, 3, 5, 6, 9, 10), 0.47543916853474627]
[(2, 3, 5, 6, 9, 11), 0.4911982881597718]
[(2, 3, 5, 6, 10, 11), 0.5021438760953739]
[(2, 3, 5, 7, 8, 9), 0.5021099110114802]
[(2, 3, 5, 7, 8, 10), 0.5040146729162421]
[(2, 3, 5, 7, 8, 11), 0.45024658650906874]
[(2, 3, 5, 7, 9, 10), 0.5306840567896204]
[(2, 3, 5, 7, 9, 11), 0.5045173561578697]
[(2, 3, 5, 7, 10, 11), 0.5068989878405]
[(2, 3, 5, 8, 9, 10), 0.5368752122817744]
[(2, 3, 5, 8, 9, 11), 0.4659486447931527]
[(2, 3, 5, 8, 10, 11), 0.488330276

[(1, 2, 3, 4, 5, 9, 11), 0.4959513619998641]
[(1, 2, 3, 4, 5, 10, 11), 0.5016615719040826]
[(1, 2, 3, 4, 6, 7, 8), 0.5235425582501189]
[(1, 2, 3, 4, 6, 7, 9), 0.456398342503906]
[(1, 2, 3, 4, 6, 7, 10), 0.4444901840907547]
[(1, 2, 3, 4, 6, 7, 11), 0.5211677195842673]
[(1, 2, 3, 4, 6, 8, 9), 0.4678289518375111]
[(1, 2, 3, 4, 6, 8, 10), 0.46163779634535707]
[(1, 2, 3, 4, 6, 8, 11), 0.48117994701446914]
[(1, 2, 3, 4, 6, 9, 10), 0.5216398342503906]
[(1, 2, 3, 4, 6, 9, 11), 0.5035670131105224]
[(1, 2, 3, 4, 6, 10, 11), 0.5054717750152843]
[(1, 2, 3, 4, 7, 8, 9), 0.5197418653624074]
[(1, 2, 3, 4, 7, 8, 10), 0.4754541131716595]
[(1, 2, 3, 4, 7, 8, 11), 0.5083153318388697]
[(1, 2, 3, 4, 7, 9, 10), 0.5221153454249031]
[(1, 2, 3, 4, 7, 9, 11), 0.5059391345696623]
[(1, 2, 3, 4, 7, 10, 11), 0.5149745261870797]
[(1, 2, 3, 4, 8, 9, 10), 0.47020718701175196]
[(1, 2, 3, 4, 8, 9, 11), 0.5164105699341077]
[(1, 2, 3, 4, 8, 10, 11), 0.46355818218871]
[(1, 2, 3, 4, 9, 10, 11), 0.5035710889205897]
[(1, 2, 3

[(1, 4, 5, 7, 8, 10, 11), 0.46930099857346647]
[(1, 4, 5, 7, 9, 10, 11), 0.42597989267033487]
[(1, 4, 5, 8, 9, 10, 11), 0.47500033965083893]
[(1, 4, 6, 7, 8, 9, 10), 0.46215338631886427]
[(1, 4, 6, 7, 8, 9, 11), 0.4888193736838529]
[(1, 4, 6, 7, 8, 10, 11), 0.45977583044630127]
[(1, 4, 6, 7, 9, 10, 11), 0.4369275185109707]
[(1, 4, 6, 8, 9, 10, 11), 0.5050003396508389]
[(1, 4, 7, 8, 9, 10, 11), 0.472627538890021]
[(1, 5, 6, 7, 8, 9, 10), 0.4083384280959174]
[(1, 5, 6, 7, 8, 9, 11), 0.48121459140004075]
[(1, 5, 6, 7, 8, 10, 11), 0.4816948576862985]
[(1, 5, 6, 7, 9, 10, 11), 0.4740499966034916]
[(1, 5, 6, 8, 9, 10, 11), 0.4983302764757829]
[(1, 5, 7, 8, 9, 10, 11), 0.49167855444602954]
[(1, 6, 7, 8, 9, 10, 11), 0.4931084844779567]
[(2, 3, 4, 5, 6, 7, 8), 0.5154479994565587]
[(2, 3, 4, 5, 6, 7, 9), 0.48305685755043815]
[(2, 3, 4, 5, 6, 7, 10), 0.48210243869302355]
[(2, 3, 4, 5, 6, 7, 11), 0.5030860675225868]
[(2, 3, 4, 5, 6, 8, 9), 0.4854289790095781]
[(2, 3, 4, 5, 6, 8, 10), 0.47210311799

[(1, 2, 3, 5, 6, 8, 9, 11), 0.5126044426329733]
[(1, 2, 3, 5, 6, 8, 10, 11), 0.5111724746960126]
[(1, 2, 3, 5, 6, 9, 10, 11), 0.5035751647306569]
[(1, 2, 3, 5, 7, 8, 9, 10), 0.48021465933020857]
[(1, 2, 3, 5, 7, 8, 9, 11), 0.4559438896814075]
[(1, 2, 3, 5, 7, 8, 10, 11), 0.49307927450580796]
[(1, 2, 3, 5, 7, 9, 10, 11), 0.5030758779974186]
[(1, 2, 3, 5, 8, 9, 10, 11), 0.49642143876095374]
[(1, 2, 3, 6, 7, 8, 9, 10), 0.4687894844100265]
[(1, 2, 3, 6, 7, 8, 9, 11), 0.4964166836492086]
[(1, 2, 3, 6, 7, 8, 10, 11), 0.5016513823789145]
[(1, 2, 3, 6, 7, 9, 10, 11), 0.505457509680049]
[(1, 2, 3, 6, 8, 9, 10, 11), 0.4997513755858977]
[(1, 2, 3, 7, 8, 9, 10, 11), 0.4626105563480742]
[(1, 2, 4, 5, 6, 7, 8, 9), 0.44832959717410503]
[(1, 2, 4, 5, 6, 7, 8, 10), 0.45214863120711907]
[(1, 2, 4, 5, 6, 7, 8, 11), 0.4764343454928333]
[(1, 2, 4, 5, 6, 7, 9, 10), 0.4921506691121527]
[(1, 2, 4, 5, 6, 7, 9, 11), 0.458819373683853]
[(1, 2, 4, 5, 6, 7, 10, 11), 0.49404252428503503]
[(1, 2, 4, 5, 6, 8, 9, 10),

[(1, 3, 5, 6, 7, 8, 9, 10, 11), 0.49642075945927583]
[(1, 4, 5, 6, 7, 8, 9, 10, 11), 0.48406901705047206]
[(2, 3, 4, 5, 6, 7, 8, 9, 10), 0.502122138441682]
[(2, 3, 4, 5, 6, 7, 8, 9, 11), 0.5035683717138781]
[(2, 3, 4, 5, 6, 7, 8, 10, 11), 0.5064207594592759]
[(2, 3, 4, 5, 6, 7, 9, 10, 11), 0.4921343658718837]
[(2, 3, 4, 5, 6, 8, 9, 10, 11), 0.5106928877114326]
[(2, 3, 4, 5, 7, 8, 9, 10, 11), 0.5083228041573262]
[(2, 3, 4, 6, 7, 8, 9, 10, 11), 0.4897527341892534]
[(2, 3, 5, 6, 7, 8, 9, 10, 11), 0.4887922016167379]
[(2, 4, 5, 6, 7, 8, 9, 10, 11), 0.48835269343115284]
[(3, 4, 5, 6, 7, 8, 9, 10, 11), 0.46166836492086133]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), 0.5130704435839957]
[(1, 2, 3, 4, 5, 6, 7, 8, 9, 11), 0.45832959717410504]
[(1, 2, 3, 4, 5, 6, 7, 8, 10, 11), 0.5045221112696149]
[(1, 2, 3, 4, 5, 6, 7, 9, 10, 11), 0.4088044290469397]
[(1, 2, 3, 4, 5, 6, 8, 9, 10, 11), 0.418324162760682]
[(1, 2, 3, 4, 5, 7, 8, 9, 10, 11), 0.497856123904626]
[(1, 2, 3, 4, 6, 7, 8, 9, 10, 11), 0.49595543780

In [15]:
N=max([tup[1] for tup in mlp_data_poi])
mlp_best_subs=[tupe[0] for tupe in mlp_data_poi if tupe[1]==N]

In [16]:
N

0.5435296515182393

In [17]:
mlp_best_subs

[(2, 3, 4, 5, 8, 10)]

In [25]:
#for whatever reason, this takes FOREVER to run
candid=[]
candid.extend(mlp_can)
for i in (2, 3, 4, 5, 8, 10):
    candid.extend(mlp_keeps[i-1])
Xqq=ufc_best_df[method_mask][candid].iloc[0:2101]
yqq=ufc_fights_clean_stance[method_mask].iloc[0:2101]['method']

In [29]:
time cross_val_score(mlp,Xqq,yqq,cv=6).mean()

CPU times: user 2min 14s, sys: 1.03 s, total: 2min 15s
Wall time: 1min 8s


0.5578105865975971

In [30]:
time cross_val_score(mlp,Xqq,yqq,cv=7).mean()

CPU times: user 1min 43s, sys: 898 ms, total: 1min 43s
Wall time: 53.1 s


0.5468255222011256

In [31]:
time cross_val_score(mlp,Xqq,yqq,cv=8).mean()

CPU times: user 2min 59s, sys: 1.35 s, total: 3min
Wall time: 1min 30s


0.5444341763975138

In [32]:
time cross_val_score(mlp,Xqq,yqq,cv=9).mean()

CPU times: user 4min 3s, sys: 2.19 s, total: 4min 5s
Wall time: 2min 5s


0.5578261812547876

In [33]:
decmask=ufc_fights_clean_stance[method_mask].iloc[0:2101]['method']=='DEC'

In [38]:
#this is the score of the model that always picks DEC
decmask.sum()/2101.0

0.5016658733936221

In [ ]:
#This is interesting. Precisely half of the fights result in a decision. So to beat this method, you absolutely
#have to beat a score of .5. I am only barely beating this at .55, but lets search subsets and see if we can improve

In [40]:
ultimate_mlp=[]
for i in (2, 3, 4, 5, 6, 8, 11):
    ultimate_mlp.append(mlp_maybies[i-1])
for i in (2, 3, 4, 5, 8, 10):
    ultimate_mlp.append(mlp_keeps[i-1])

In [41]:
#we have 13 but can only reasonably do 2^11 so we must join some
ultimate_mlp

[['fighter_L5Y_ko_wins_diff_2',
  'fighter_L5Y_ko_losses_diff_2',
  'fighter_L2Y_ko_wins_diff_2',
  'fighter_L2Y_ko_losses_diff_2'],
 ['fighter_sub_losses_diff_2',
  'fighter_L5Y_sub_wins_diff_2',
  'fighter_L5Y_sub_losses_diff_2',
  'fighter_L2Y_sub_wins_diff_2',
  'fighter_L2Y_sub_losses_diff_2'],
 ['fighter_inf_reversals_avg',
  'fighter_abs_reversals_avg',
  'opponent_inf_reversals_avg',
  'opponent_abs_reversals_avg'],
 ['fighter_inf_sub_attempts_avg',
  'fighter_abs_sub_attempts_avg',
  'opponent_inf_sub_attempts_avg',
  'opponent_abs_sub_attempts_avg'],
 ['fighter_inf_takedowns_attempts_avg',
  'fighter_abs_takedowns_attempts_avg',
  'opponent_inf_takedowns_attempts_avg',
  'opponent_abs_takedowns_attempts_avg'],
 ['fighter_inf_total_strikes_attempts_avg',
  'fighter_abs_total_strikes_attempts_avg',
  'opponent_inf_total_strikes_attempts_avg',
  'opponent_abs_total_strikes_attempts_avg'],
 ['fighter_inf_clinch_strikes_landed_avg',
  'fighter_abs_clinch_strikes_landed_avg',
  'op

In [42]:
#joined takedowns with pass, kowins diff with L5Y ko wins, sub wins diff with L5Y sub wins...
mlp_ultimate_modified=[
['fighter_L5Y_ko_wins_diff_2',
  'fighter_L5Y_ko_losses_diff_2',
  'fighter_L2Y_ko_wins_diff_2',
  'fighter_L2Y_ko_losses_diff_2',
  'fighter_ko_wins_diff_2',
  'fighter_ko_losses_diff_2',],
 ['fighter_sub_losses_diff_2',
  'fighter_L5Y_sub_wins_diff_2',
  'fighter_L5Y_sub_losses_diff_2',
  'fighter_L2Y_sub_wins_diff_2',
  'fighter_L2Y_sub_losses_diff_2'],
 ['fighter_inf_reversals_avg',
  'fighter_abs_reversals_avg',
  'opponent_inf_reversals_avg',
  'opponent_abs_reversals_avg'],
 ['fighter_inf_sub_attempts_avg',
  'fighter_abs_sub_attempts_avg',
  'opponent_inf_sub_attempts_avg',
  'opponent_abs_sub_attempts_avg',
  'fighter_sub_wins_diff_2'],
 ['fighter_inf_takedowns_attempts_avg',
  'fighter_abs_takedowns_attempts_avg',
  'opponent_inf_takedowns_attempts_avg',
  'opponent_abs_takedowns_attempts_avg'],
 ['fighter_inf_total_strikes_attempts_avg',
  'fighter_abs_total_strikes_attempts_avg',
  'opponent_inf_total_strikes_attempts_avg',
  'opponent_abs_total_strikes_attempts_avg'],
 ['fighter_inf_clinch_strikes_landed_avg',
  'fighter_abs_clinch_strikes_landed_avg',
  'opponent_inf_clinch_strikes_landed_avg',
  'opponent_abs_clinch_strikes_landed_avg'],
 ['fighter_inf_head_strikes_landed_avg',
  'fighter_abs_head_strikes_landed_avg',
  'opponent_inf_head_strikes_landed_avg',
  'opponent_abs_head_strikes_landed_avg'],
 ['fighter_inf_body_strikes_landed_avg',
  'fighter_inf_body_strikes_attempts_avg',
  'fighter_abs_body_strikes_landed_avg',
  'fighter_abs_body_strikes_attempts_avg',
  'opponent_inf_body_strikes_landed_avg',
  'opponent_inf_body_strikes_attempts_avg',
  'opponent_abs_body_strikes_landed_avg',
  'opponent_abs_body_strikes_attempts_avg'],
 ['fighter_inf_takedowns_landed_avg',
  'fighter_abs_takedowns_landed_avg',
  'opponent_inf_takedowns_landed_avg',
  'opponent_abs_takedowns_landed_avg',
  'fighter_inf_pass_avg',
  'fighter_abs_pass_avg',
  'opponent_inf_pass_avg',
  'opponent_abs_pass_avg'],
 ['fighter_inf_leg_strikes_landed_avg',
  'fighter_inf_leg_strikes_attempts_avg',
  'fighter_abs_leg_strikes_landed_avg',
  'fighter_abs_leg_strikes_attempts_avg',
  'opponent_inf_leg_strikes_landed_avg',
  'opponent_inf_leg_strikes_attempts_avg',
  'opponent_abs_leg_strikes_landed_avg',
  'opponent_abs_leg_strikes_attempts_avg'],
]
len(mlp_ultimate_modified)

11

In [ ]:
mlp=MLPClassifier(max_iter=2000,alpha=.001,solver='sgd',random_state=3)
yqq=ufc_fights_clean_stance[method_mask].iloc[0:2101]['method']
mlp_data_poi_mod=[]
SUB=subsets(11)
for S in SUB:
    candids=[]
    if len(S)==0:
        continue
    for i in S:
        candids.extend(mlp_ultimate_modified[i-1])
    Xqq=ufc_best_df[method_mask][candids].iloc[0:2101]
    point=[S,cross_val_score(mlp,Xqq,yqq,cv=3).mean()]
    mlp_data_poi_mod.append(point)
    print(point)

[(1,), 0.5482956320902113]
[(2,), 0.49310237076285574]
[(3,), 0.5016656477141499]
[(4,), 0.49737721622172404]
[(5,), 0.5016656477141499]
[(6,), 0.4302357176822227]
[(7,), 0.5016656477141499]
[(8,), 0.4959513619998641]
[(9,), 0.5026180286665308]
[(10,), 0.5016656477141499]
[(11,), 0.5016656477141499]


/opt/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[(1, 2), 0.568765029549623]
[(1, 3), 0.5573432511378303]
[(1, 4), 0.5582888390734325]
[(1, 5), 0.5592480130425922]
[(1, 6), 0.45593030364785]
[(1, 7), 0.5582935941851775]
[(1, 8), 0.5335337273283065]
[(1, 9), 0.5554398478364242]
[(1, 10), 0.5487684260580123]
[(1, 11), 0.5559207934243598]
[(2, 3), 0.4945295835880715]
[(2, 4), 0.5030928605393655]
[(2, 5), 0.49976564092113307]
[(2, 6), 0.4883472590177298]
[(2, 7), 0.49928877114326475]
[(2, 8), 0.4754928333672985]
[(2, 9), 0.4926255009849874]
[(2, 10), 0.4864370626995449]
[(2, 11), 0.4802520209224917]
[(3, 4), 0.5002343590788669]
[(3, 5), 0.5016656477141499]
[(3, 6), 0.334999660349161]
[(3, 7), 0.5016656477141499]
[(3, 8), 0.5002404727939678]
[(3, 9), 0.501664968412472]
[(3, 10), 0.5002370762855785]
[(3, 11), 0.5011914951429931]
[(4, 5), 0.501186740031248]
[(4, 6), 0.45500441546090614]


In [ ]:
#This is good! do this with the above one too...